In [28]:
# Cell 1: Initialize Binance Client for Binance.US (Revised)

import os
import logging
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceRequestException
from dotenv import load_dotenv

# --- Configuration ---
# Load environment variables from a .env file if present
load_dotenv()

# Ensure your API keys are stored securely, e.g., as environment variables
API_KEY = os.environ.get('BINANCE_API_KEY')
SECRET_KEY = os.environ.get('BINANCE_SECRET_KEY') # Using SECRET_KEY as in my original example, adjust if you use BINANCE_API_SECRET
TARGET_TLD = 'us' # Use 'us' for Binance.US

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Initialize Client ---
client = None
logging.info("--- Initializing Binance Client ---")
if not API_KEY or not SECRET_KEY:
    logging.error("Binance API Key or Secret Key not found in environment variables.")
    logging.error("Please set BINANCE_API_KEY and BINANCE_SECRET_KEY in your environment or a .env file.")
else:
    try:
        client = Client(API_KEY, SECRET_KEY, tld=TARGET_TLD)
        # Test connectivity using ping
        client.ping()
        logging.info(f"✅ Binance client initialized and connection verified successfully for tld='{TARGET_TLD}'.")

        # Optional: Perform a quick check to confirm account access
        account_status = client.get_account_status()
        logging.info(f"Account status check: {account_status.get('data', 'N/A')}")

    except BinanceAPIException as e:
        logging.error(f"❌ Binance API Exception during initialization or ping: {e}")
        client = None # Ensure client is None if connection failed
    except BinanceRequestException as e:
        logging.error(f"❌ Binance Request Exception during initialization or ping: {e}")
        client = None # Ensure client is None if connection failed
    except Exception as e:
        logging.error(f"❌ An unexpected error occurred during client initialization: {e}")
        client = None # Ensure client is None if connection failed

# --- Verification ---
if client:
    logging.info("Binance client is ready.")
else:
    logging.warning("Binance client initialization failed. Cannot proceed with API calls.")
    # You might want to raise an error or exit here depending on your script's needs
    # raise RuntimeError("Client initialization failed.")

# End of Cell 1

2025-04-09 20:31:29,040 - INFO - --- Initializing Binance Client ---
2025-04-09 20:31:29,041 - ERROR - Binance API Key or Secret Key not found in environment variables.
2025-04-09 20:31:29,042 - ERROR - Please set BINANCE_API_KEY and BINANCE_SECRET_KEY in your environment or a .env file.
2025-04-09 20:31:29,042 - WARNING - Binance client initialization failed. Cannot proceed with API calls.


In [2]:
# Cell 2: Initialize Client & Get Comprehensive Portfolio State

import pandas as pd
import os
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN # Use Decimal for precision
from datetime import datetime, timezone # Import timezone
import warnings

# Suppress specific Pandas warnings if desired, use cautiously
# warnings.simplefilter(action='ignore', category=FutureWarning)

# --- Configuration ---
QUOTE_ASSET = 'USDT' # Define your quote asset
DISPLAY_PRECISION = 8 # How many decimal places to show in output

# --- Initialize Binance Client ---
print("--- Initializing Binance Client ---")
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')
client = None
if api_key and api_secret:
    try:
        # Ensure tld='us' for Binance.US
        client = Client(api_key, api_secret, tld='us')
        client.ping()
        print("✅ Binance client initialized successfully.")
    except Exception as e:
        print(f"❌ Error initializing Binance client: {e}")
        # Stop execution if client fails to initialize
        raise RuntimeError(f"Client initialization failed: {e}")
else:
    # Stop execution if keys are missing
    raise ValueError("❌ API Key or Secret not found in environment variables.")

# --- State Fetching Functions (Define or Verify from Cell 1) ---
# These functions should ideally be defined in Cell 1. We check for their existence.
# If Cell 1 only contained helpers pasted from history, we might need to define them here.
# For now, assume they are defined correctly in Cell 1's execution context.
if 'get_current_balances_detailed' not in locals() or 'get_open_orders_detailed' not in locals():
    # If the functions weren't defined in Cell 1 for some reason, define them here.
    # This provides robustness but defining in Cell 1 is cleaner.
    print("⚠️ State fetching functions not found in local scope, defining them now...")

    def get_current_balances_detailed(api_client):
        """ Fetches detailed balances using Decimal. """
        if not api_client: return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)
        try:
            account_info = api_client.get_account(); balances_raw = account_info.get('balances', [])
            processed = [{'Asset':i['asset'],'Free':Decimal(i['free']),'Locked':Decimal(i['locked'])} for i in balances_raw if Decimal(i['free'])>0 or Decimal(i['locked'])>0]
            if not processed: return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)
            df = pd.DataFrame(processed); df.set_index('Asset', inplace=True); return df
        except Exception as e: print(f"❌ Error balances: {e}"); return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)

    def get_open_orders_detailed(api_client):
        """ Fetches detailed open orders using Decimal. """
        if not api_client: return pd.DataFrame()
        try:
            orders = api_client.get_open_orders()
            if not orders: return pd.DataFrame()
            df = pd.DataFrame(orders)
            for col in ['price','origQty','executedQty','cummulativeQuoteQty','stopPrice']:
                if col in df.columns: df[col] = df[col].apply(lambda x: Decimal(str(x)) if x is not None else Decimal('0'))
            for col in ['time','updateTime']:
                if col in df.columns: df[col] = pd.to_datetime(df[col], unit='ms', utc=True, errors='coerce')
            return df.sort_values(by=['symbol', 'time'], ascending=[True, False]).reset_index(drop=True)
        except Exception as e: print(f"❌ Error orders: {e}"); return pd.DataFrame()


# --- Main Execution Logic for this Cell ---
print("\n--- Fetching Comprehensive Portfolio State ---")
portfolio_state = {
    "balances": pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object),
    "open_orders": pd.DataFrame()
}

try:
    # Call the functions to get current state
    portfolio_state["balances"] = get_current_balances_detailed(client)
    portfolio_state["open_orders"] = get_open_orders_detailed(client)

    print("\n--- Portfolio State Summary ---")

    # --- Format Balances DataFrame for Display ---
    if not portfolio_state["balances"].empty:
        print("Balances (Free/Locked):")
        balances_display_df = portfolio_state["balances"].copy()
        formatter = f'{{:.{DISPLAY_PRECISION}f}}'
        balances_display_df['Free'] = balances_display_df['Free'].apply(lambda x: formatter.format(x) if isinstance(x, Decimal) else x)
        balances_display_df['Locked'] = balances_display_df['Locked'].apply(lambda x: formatter.format(x) if isinstance(x, Decimal) else x)
        print(balances_display_df)
    else:
        print("Balances: None found or error fetching.")

    # --- Format Open Orders DataFrame for Display ---
    if not portfolio_state["open_orders"].empty:
        print(f"\nOpen Orders ({len(portfolio_state['open_orders'])}):")
        orders_display_df = portfolio_state["open_orders"].copy()
        cols_to_format = ['price', 'origQty', 'executedQty']
        cols_to_display = ['symbol','orderId','side','type','status','price','origQty','executedQty','time']
        cols_to_display = [c for c in cols_to_display if c in orders_display_df.columns]
        formatter = f'{{:.{DISPLAY_PRECISION}f}}'
        for col in cols_to_format:
            if col in orders_display_df.columns:
                 orders_display_df[col] = orders_display_df[col].apply(lambda x: formatter.format(x) if isinstance(x, Decimal) else x)
        # Convert time to a more readable string format for display if needed
        if 'time' in orders_display_df.columns:
            orders_display_df['time_str'] = orders_display_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S') # Add readable time string
            cols_to_display.append('time_str') # Add it to display columns
            cols_to_display.remove('time') # Remove original datetime object from display

        print(orders_display_df[cols_to_display])
    else:
        print("\nOpen Orders: None found or error fetching.")

    print("\n✅ State captured in 'portfolio_state' dictionary.")

except Exception as state_fetch_error:
     print(f"\n❌❌ An error occurred during state fetching: {state_fetch_error}")
     # Ensure portfolio_state is defined but indicates failure
     portfolio_state = {"balances": pd.DataFrame(), "open_orders": pd.DataFrame(), "error": str(state_fetch_error)}
     print("   Subsequent steps may fail.")


# Result: 'portfolio_state' dictionary holds the initial balances and open orders.

--- Initializing Binance Client ---
✅ Binance client initialized successfully.
⚠️ State fetching functions not found in local scope, defining them now...

--- Fetching Comprehensive Portfolio State ---

--- Portfolio State Summary ---
Balances (Free/Locked):
             Free      Locked
Asset                        
BTC    0.00005710  0.00000000
ETH    0.00318720  0.00000000
XRP    5.97600000  0.00000000
USDT   8.96644535  0.00000000
ADA    1.99200000  0.00000000
BUSD   0.14956400  0.00000000
WAVES  0.01000000  0.00000000
SOL    0.02988000  0.00000000

Open Orders: None found or error fetching.

✅ State captured in 'portfolio_state' dictionary.


In [3]:
# Cell 3 (NEW): Configure LLM (Google AI)

import os
from dotenv import load_dotenv
try:
    import google.generativeai as genai
    google_ai_package_found = True
except ImportError:
    print("WARNING: google.generativeai package not found. Install with: pip install google-generativeai")
    google_ai_package_found = False
    genai = None # Ensure genai exists but is None

print("\n--- Configuring LLM (Google AI) ---")
genai_configured = False # Default state

if google_ai_package_found:
    load_dotenv()
    google_api_key = os.environ.get('GOOGLE_API_KEY')
    if not google_api_key:
        print("⚠️ GOOGLE_API_KEY not found in environment variables. LLM features will be skipped.")
    else:
        try:
            genai.configure(api_key=google_api_key)
            genai_configured = True # Set flag on success
            print("✅ Google AI configured successfully.")
        except Exception as e:
            print(f"❌ Error configuring Google AI: {e}")
            print("   LLM features will likely fail.")
else:
    print("Skipping configuration because google-generativeai package is missing.")

# Ensure variable exists for subsequent checks
if 'genai_configured' not in locals(): genai_configured = False


--- Configuring LLM (Google AI) ---
✅ Google AI configured successfully.


In [4]:
# Cell 4: Identify Top N Assets by Volume

import pandas as pd
import os
from decimal import Decimal # Use Decimal for price/volume if needed later

# --- Configuration ---
N = 10 # Number of top crypto assets to target
QUOTE_ASSET_FILTER = 'USDT' # Should match QUOTE_ASSET usually
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD', 'TUSD', 'USDP', 'FDUSD'] # Define stablecoins to exclude
VOLUME_DISPLAY_PRECISION = 2 # Decimals for volume display if printing details

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client is not initialized. Please run Cell 2 first.")

# --- Initialize ---
top_n_assets = []
top_n_symbols = []
volume_ranking_df = pd.DataFrame() # Will store detailed volume info

# --- Main Logic ---
print(f"\n--- Identifying Top {N} Assets by 24hr {QUOTE_ASSET_FILTER} Volume ---")
try:
    all_tickers = client.get_ticker() # Fetch all symbol tickers
    # Filter for pairs ending with the quote asset (e.g., 'USDT')
    usdt_tickers = [t for t in all_tickers if t.get('symbol', '').endswith(QUOTE_ASSET_FILTER)]
    print(f"Processing {len(usdt_tickers)} {QUOTE_ASSET_FILTER} pairs...")

    volume_data = []
    for ticker in usdt_tickers:
        symbol = ticker.get('symbol')
        if not symbol: continue # Skip if symbol key is missing

        # Derive base asset (e.g., 'BTC' from 'BTCUSDT')
        base_asset = symbol[:-len(QUOTE_ASSET_FILTER)] # More robust than replace

        # Skip if the base asset is a known stablecoin
        if base_asset in STABLECOIN_ASSETS: continue

        try:
             # Extract quote volume and last price, convert safely to Decimal
             volume_str = ticker.get('quoteVolume', '0.0')
             price_str = ticker.get('lastPrice', '0.0')
             volume = Decimal(volume_str)
             last_price = Decimal(price_str)

             # Only include assets with positive volume and price
             if volume > 0 and last_price > 0:
                  volume_data.append({
                      'Asset': base_asset,
                      'Symbol': symbol,
                      f'Volume_{QUOTE_ASSET_FILTER}': volume, # Store as Decimal
                      'LastPrice': last_price # Store as Decimal
                  })
        except Exception as e:
             # Log ticker processing errors if needed, otherwise skip problematic tickers
             # print(f"  Warning: Skipping {symbol} due to data error: {e}")
             pass # Continue to the next ticker

    if not volume_data:
         print("⚠️ Warning: No valid volume data found after filtering.")
    else:
         # Create DataFrame from collected data
         volume_ranking_df = pd.DataFrame(volume_data)
         # Sort by volume (descending)
         volume_ranking_df.sort_values(by=f'Volume_{QUOTE_ASSET_FILTER}', ascending=False, inplace=True)
         # Select the top N rows
         top_n_df = volume_ranking_df.head(N)
         # Extract asset names and symbols
         top_n_assets = top_n_df['Asset'].tolist()
         top_n_symbols = top_n_df['Symbol'].tolist()
         print(f"Top {N} Assets (by USDT Volume): {top_n_assets}") # List print is fine

         # --- Optional: Print DataFrame with Corrected Formatting ---
         # print(f"\n--- Top {N} Volume Details ---")
         # display_format = f'{{:,.{VOLUME_DISPLAY_PRECISION}f}}' # Format with commas, fixed decimals
         # cols_to_show = ['Asset', 'Symbol', f'Volume_{QUOTE_ASSET_FILTER}']
         # with pd.option_context('display.float_format', display_format.format):
         #    top_n_display = top_n_df[cols_to_show].copy()
         #    print(top_n_display)
         # --- End Optional Print ---

except Exception as e:
    print(f"❌ An error occurred fetching/processing ticker data: {e}")
    # Reset variables to ensure clean state on error
    top_n_assets, top_n_symbols, volume_ranking_df = [], [], pd.DataFrame()

# --- Final Check ---
if not top_n_symbols:
    print("⚠️ Top N symbols list is empty. Subsequent steps might fail.")
    # Consider raising an error if this is critical for the strategy
    # raise ValueError("Failed to identify Top N symbols.")
else:
    print(f"✅ Top {len(top_n_symbols)} symbols identified: {top_n_symbols}")
    print(f"   Volume ranking data stored in 'volume_ranking_df'.")


--- Identifying Top 10 Assets by 24hr USDT Volume ---
Processing 188 USDT pairs...
Top 10 Assets (by USDT Volume): ['XRP', 'BTC', 'SOL', 'ETH', 'ADA', 'LTC', 'DOGE', 'HBAR', 'SUI', 'XLM']
✅ Top 10 symbols identified: ['XRPUSDT', 'BTCUSDT', 'SOLUSDT', 'ETHUSDT', 'ADAUSDT', 'LTCUSDT', 'DOGEUSDT', 'HBARUSDT', 'SUIUSDT', 'XLMUSDT']
   Volume ranking data stored in 'volume_ranking_df'.


In [5]:
# Cell 5: Fetch/Load Historical Data for Top N Symbols

import pandas as pd
import os
from binance.client import Client # For interval constants
from datetime import datetime, timezone # Import timezone

# --- Configuration ---
INTERVALS_TO_PROCESS = ['1d', '1h', '5m', '1m'] # Timeframes needed for analysis
# Map user-friendly keys to Binance API constants (ensure Client is imported)
INTERVAL_API_MAP = {
    '1d': Client.KLINE_INTERVAL_1DAY, '1h': Client.KLINE_INTERVAL_1HOUR,
    '5m': Client.KLINE_INTERVAL_5MINUTE, '1m': Client.KLINE_INTERVAL_1MINUTE
}
# Define fetch ranges using relative time strings (used if fetching fresh data)
START_DATES_FETCH = {
    '1d': "90 days ago UTC", '1h': "7 days ago UTC",
    '5m': "3 days ago UTC", '1m': "1 day ago UTC"
}
# Define identifiers used in filenames when saving/loading CSV data
START_DATES_SAVE_IDS = {
    '1d': "90_days_ago_UTC", '1h': "7_days_ago_UTC",
    '5m': "3_days_ago_UTC", '1m': "1_day_ago_UTC"
}
DATA_DIRECTORY = "data" # Subdirectory to store/load CSV files
FORCE_FETCH_FRESH = False # Set True to always ignore local CSVs and fetch from API

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client not initialized. Run Cell 2 first.")
# Ensure fetch helper function exists (should be from Cell 1)
if 'fetch_and_process_klines' not in locals():
     raise RuntimeError("Helper function 'fetch_and_process_klines' not defined. Run Cell 1.")
if 'top_n_symbols' not in locals() or not top_n_symbols:
    raise ValueError("'top_n_symbols' list not found or empty. Run Cell 3.")

# --- Initialization ---
top_n_klines = {} # Dictionary structure: {'SYMBOL': {'interval': DataFrame}}
os.makedirs(DATA_DIRECTORY, exist_ok=True) # Ensure data directory exists
fetch_errors = 0
load_errors = 0
save_errors = 0

print(f"\n--- Processing Kline Data for Top {len(top_n_symbols)} Symbols ---")
print(f"Intervals: {INTERVALS_TO_PROCESS}. Will {'FETCH FRESH' if FORCE_FETCH_FRESH else 'LOAD local first'}.")

# Loop through each symbol identified in Cell 3
for symbol in top_n_symbols:
    top_n_klines[symbol] = {} # Initialize nested dictionary for the symbol
    symbol_fetch_errors = 0

    # Loop through each required interval for the current symbol
    for interval_key in INTERVALS_TO_PROCESS:
        interval_value = INTERVAL_API_MAP.get(interval_key)
        if not interval_value:
             print(f"  ⚠️ Skipping invalid interval key: {interval_key}")
             continue # Skip if interval key isn't mapped

        df = None # Reset DataFrame for each interval
        loaded_from_csv = False
        source_msg = "" # To track if loaded or fetched

        # --- 1. Attempt to Load from CSV ---
        if not FORCE_FETCH_FRESH:
            start_id = START_DATES_SAVE_IDS.get(interval_key)
            if start_id:
                filename = os.path.join(DATA_DIRECTORY, f"{symbol}_{interval_key}_{start_id}.csv")
                if os.path.exists(filename):
                    try:
                        # Read CSV, set index, parse dates
                        df = pd.read_csv(filename, index_col='Open Time', parse_dates=True)

                        # Corrected Timezone Handling
                        if df.index.tz is None:
                            df.index = df.index.tz_localize('UTC')
                        elif df.index.tz != timezone.utc:
                             print(f"  Warning: Timezone for {filename} is {df.index.tz}, converting to UTC.")
                             df.index = df.index.tz_convert('UTC')

                        # If df is still valid after timezone check/conversion
                        if df is not None:
                             loaded_from_csv = True
                             source_msg = f"Loaded {symbol} {interval_key} from CSV."

                    except Exception as e:
                        print(f"  ⚠️ Error loading {filename}: {e}. Will try fetching.")
                        load_errors += 1
                        df = None # Ensure df is None if loading fails

        # --- 2. Fetch Fresh if Not Loaded or if Forcing ---
        if df is None: # Fetch if df is None (due to not existing, load error, or FORCE_FETCH_FRESH)
            start_str = START_DATES_FETCH.get(interval_key, "7 days ago UTC")
            print(f"  Fetching {symbol} {interval_key} data starting {start_str}...")
            # Call the helper function defined in Cell 1
            df = fetch_and_process_klines(client, symbol, interval_value, start_str)
            source_msg = f"Fetched {symbol} {interval_key} from API."

            # --- 3. Optionally Save Freshly Fetched Data ---
            if df is not None and not df.empty:
                 save_id = START_DATES_SAVE_IDS.get(interval_key)
                 if save_id:
                      save_filename = os.path.join(DATA_DIRECTORY, f"{symbol}_{interval_key}_{save_id}.csv")
                      try:
                           df.to_csv(save_filename)
                           source_msg += " Saved to CSV."
                      except Exception as e:
                           print(f"  ⚠️ Error SAVING {save_filename}: {e}")
                           save_errors += 1
                 else:
                      print(f"  ⚠️ Cannot determine save filename for {interval_key}. Data not saved.")
            elif df is None: # Indicates an error during fetch/process reported by helper
                 symbol_fetch_errors += 1
                 source_msg += " Fetch/Process FAILED."


        # --- 4. Store the final DataFrame ---
        if df is not None and not df.empty:
             top_n_klines[symbol][interval_key] = df
        elif not loaded_from_csv: # Only flag error if fetch failed
             print(f"  ⚠️ No data obtained or processed for {symbol} {interval_key}.")


    # --- End of interval loop ---
    if symbol_fetch_errors > 0:
        fetch_errors += symbol_fetch_errors

# --- End of symbol loop ---

# --- Final Summary ---
if load_errors > 0: print(f"⚠️ Encountered {load_errors} error(s) loading existing CSV data.")
if fetch_errors > 0: print(f"⚠️ Encountered {fetch_errors} error(s) fetching/processing fresh data.")
if save_errors > 0: print(f"⚠️ Encountered {save_errors} error(s) saving fetched data.")

missing_data_symbols = []
for symbol, intervals in top_n_klines.items():
     loaded_intervals = list(intervals.keys())
     if len(loaded_intervals) < len(INTERVALS_TO_PROCESS):
          missing = set(INTERVALS_TO_PROCESS) - set(loaded_intervals)
          missing_data_symbols.append(f"{symbol} (missing: {', '.join(missing)})")
if missing_data_symbols:
     print(f"⚠️ Symbols potentially missing required interval data: {'; '.join(missing_data_symbols)}")

print(f"✅ Kline data processing complete. Data stored in 'top_n_klines' dictionary.")


--- Processing Kline Data for Top 10 Symbols ---
Intervals: ['1d', '1h', '5m', '1m']. Will LOAD local first.
✅ Kline data processing complete. Data stored in 'top_n_klines' dictionary.


In [6]:
# Cell 6: Calculate Technical Indicators for Top N Symbols

import pandas as pd
import numpy as np # Needed for manual calculations

# --- Configuration ---
SMA_PERIODS = [20, 50]      # Periods for Simple Moving Averages
RSI_PERIOD = 14             # Period for Relative Strength Index
MACD_FAST = 12              # Fast EMA period for MACD
MACD_SLOW = 26              # Slow EMA period for MACD
MACD_SIGNAL = 9             # Signal Line EMA period for MACD

# --- Check Prerequisites ---
if 'top_n_klines' not in locals() or not top_n_klines:
    raise ValueError("'top_n_klines' dictionary not found or empty. Run Cell 4 first.")

print("\n--- Calculating Technical Indicators ---")
calculation_errors = 0

# Loop through each symbol in the main dictionary
for symbol, interval_dict in top_n_klines.items():
    symbol_errors = 0
    # Loop through each interval ('1d', '1h', etc.) for the current symbol
    for interval, df in interval_dict.items():
        # Ensure DataFrame is not empty and has the 'Close' column needed
        if df.empty or 'Close' not in df.columns:
            # Silently skip empty DFs or those missing essential data
            continue

        try:
            # --- Calculate SMAs ---
            for period in SMA_PERIODS:
                # Ensure enough data points for the rolling window
                if len(df) >= period:
                    # Use min_periods=period to avoid partial calculations at the start
                    df[f'SMA_{period}'] = df['Close'].rolling(window=period, min_periods=period).mean()
                else:
                    df[f'SMA_{period}'] = np.nan # Assign NaN if not enough data

            # --- Calculate RSI ---
            if len(df) >= RSI_PERIOD + 1: # Need at least period+1 for diff()
                delta = df['Close'].diff()
                gain = delta.where(delta > 0, 0.0)
                loss = -delta.where(delta < 0, 0.0)
                # Use Exponential Moving Average for RSI smoothing
                avg_gain = gain.ewm(com=RSI_PERIOD - 1, min_periods=RSI_PERIOD).mean()
                avg_loss = loss.ewm(com=RSI_PERIOD - 1, min_periods=RSI_PERIOD).mean()

                # Calculate Relative Strength (RS) - handle division by zero
                rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss) # Avoid division by zero warning

                # Calculate RSI
                rsi = 100.0 - (100.0 / (1.0 + rs))
                rsi[rs == np.inf] = 100.0 # Set RSI to 100 where avg_loss was 0 (infinite RS)

                # Fill initial NaNs, potentially assigning a neutral 50.
                rsi = pd.Series(rsi, index=df.index).fillna(50.0)

                df[f'RSI_{RSI_PERIOD}'] = rsi
            else:
                df[f'RSI_{RSI_PERIOD}'] = np.nan # Assign NaN if not enough data


            # --- Calculate MACD ---
            # Need enough data for slow EMA calculation
            if len(df) >= MACD_SLOW:
                ema_fast = df['Close'].ewm(span=MACD_FAST, adjust=False).mean()
                ema_slow = df['Close'].ewm(span=MACD_SLOW, adjust=False).mean()
                macd_line = ema_fast - ema_slow
                # Need enough data points for signal line calculation on macd_line
                if len(macd_line.dropna()) >= MACD_SIGNAL:
                     signal_line = macd_line.ewm(span=MACD_SIGNAL, adjust=False).mean()
                     histogram = macd_line - signal_line
                else:
                     signal_line = np.nan
                     histogram = np.nan

                df[f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = macd_line
                df[f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = signal_line
                df[f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = histogram
            else:
                 # Assign NaN if not enough data for MACD calculation
                 df[f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = np.nan
                 df[f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = np.nan
                 df[f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = np.nan


            # DataFrames are modified in-place within the dictionary

        except Exception as e:
            print(f"  ❌ Error calculating indicators for {symbol} {interval}: {e}")
            symbol_errors += 1
            # Attempt cleanup of potentially partially added columns
            potential_new_cols = [f'SMA_{p}' for p in SMA_PERIODS] + \
                                 [f'RSI_{RSI_PERIOD}', f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}', \
                                  f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}', f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}']
            for col in potential_new_cols:
                 if col in df.columns:
                      try: df.drop(columns=[col], inplace=True)
                      except Exception: pass # Ignore cleanup errors

    if symbol_errors > 0:
        calculation_errors += symbol_errors

if calculation_errors > 0:
     print(f"⚠️ Indicator calculation completed with {calculation_errors} total errors.")

# --- Optional: Display sample data to verify ---
# symbol_to_show = top_n_symbols[0] if top_n_symbols else None
# interval_to_show = '1h'
# if symbol_to_show and interval_to_show in top_n_klines.get(symbol_to_show, {}):
#     print(f"\n--- Sample {symbol_to_show} {interval_to_show} Data with Indicators (Last 5 Rows) ---")
#     df_sample = top_n_klines[symbol_to_show][interval_to_show]
#     cols = ['Close', f'SMA_{SMA_PERIODS[0]}', f'SMA_{SMA_PERIODS[1]}', f'RSI_{RSI_PERIOD}', f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}']
#     cols = [c for c in cols if c in df_sample.columns]
#     with pd.option_context('display.float_format', '{:.4f}'.format): print(df_sample[cols].tail())

print("✅ Indicator calculation complete for all symbols and intervals.")


--- Calculating Technical Indicators ---
✅ Indicator calculation complete for all symbols and intervals.


In [7]:
# Cell 7: Calculate Dynamic Target Allocations

import pandas as pd
import numpy as np
from decimal import Decimal # Use Decimal for precise percentage calculations

# --- Configuration ---
# Ensure N matches the number of assets selected in Cell 3
TOP_N_ASSETS_COUNT = 10
STABLECOIN_RESERVE_PCT = Decimal('20.0') # Target reserve percentage as Decimal
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD', 'TUSD', 'USDP', 'FDUSD']
# Primary quote/stablecoin, should match filter used in Cell 3
QUOTE_ASSET = 'USDT'

# --- Check Prerequisites ---
if 'volume_ranking_df' not in locals() or volume_ranking_df.empty:
    # Attempt to regenerate volume_ranking_df if it's missing (e.g., error in Cell 3)
    print("⚠️ 'volume_ranking_df' missing, attempting regeneration from tickers...")
    try:
        # Minimal regeneration logic (assumes client exists from Cell 1/2)
        if 'client' not in locals() or client is None: raise RuntimeError("Client missing for regen")
        all_tickers = client.get_ticker()
        usdt_tickers = [t for t in all_tickers if t.get('symbol', '').endswith(QUOTE_ASSET)]
        volume_data = []
        for ticker in usdt_tickers:
            symbol=ticker.get('symbol'); base=symbol[:-len(QUOTE_ASSET)]; vol_str=ticker.get('quoteVolume','0'); price_str=ticker.get('lastPrice','0')
            if base not in STABLECOIN_ASSETS:
                try: # Convert safely
                    vol=Decimal(vol_str); price=Decimal(price_str)
                    if vol > 0 and price > 0: volume_data.append({'Asset':base,'Symbol':symbol,f'Volume_{QUOTE_ASSET}':vol})
                except Exception: pass # Skip invalid tickers
        if volume_data:
            volume_ranking_df = pd.DataFrame(volume_data)
            # Ensure Volume column is Decimal if regenerated
            volume_ranking_df[f'Volume_{QUOTE_ASSET}'] = volume_ranking_df[f'Volume_{QUOTE_ASSET}'].apply(Decimal)
            volume_ranking_df.sort_values(f'Volume_{QUOTE_ASSET}',ascending=False,inplace=True)
        else:
            volume_ranking_df = pd.DataFrame() # Ensure empty if still no data

        if volume_ranking_df.empty: raise ValueError("Could not regenerate volume data.")
        print("✅ Volume ranking regenerated.")
    except Exception as e:
        # If regeneration fails, targets cannot be calculated dynamically
        raise ValueError(f"Volume data unavailable and regeneration failed: {e}. Cannot calculate targets. Check Cell 3.")

# Check if the required volume column exists and is of Decimal type
volume_col = f'Volume_{QUOTE_ASSET}'
if volume_col not in volume_ranking_df.columns:
     raise KeyError(f"Required volume column '{volume_col}' not found in volume_ranking_df. Check Cell 3.")
# Ensure the volume column has Decimal type for accurate calculation
if not pd.api.types.is_object_dtype(volume_ranking_df[volume_col]) or \
   not all(isinstance(v, Decimal) for v in volume_ranking_df[volume_col]):
    print(f"⚠️ Converting volume column '{volume_col}' to Decimal for calculation.")
    try:
        volume_ranking_df[volume_col] = volume_ranking_df[volume_col].apply(lambda x: Decimal(str(x)))
    except Exception as conv_err:
        raise TypeError(f"Failed to convert volume column to Decimal: {conv_err}")


# --- Initialize ---
# Stores target allocation percentages as Decimals
dynamic_target_allocations_pct = {}

print("\n--- Calculating Dynamic Target Allocations ---")
try:
    # Filter out stablecoins again just in case, and get top N
    crypto_volume_df = volume_ranking_df[~volume_ranking_df['Asset'].isin(STABLECOIN_ASSETS)].copy()
    top_n_crypto_df = crypto_volume_df.head(TOP_N_ASSETS_COUNT)

    # Calculate total volume of the Top N crypto assets (ensure Decimal)
    total_top_n_volume = top_n_crypto_df[volume_col].sum()

    if total_top_n_volume <= 0:
        print("⚠️ Warning: Total volume for Top N crypto assets is zero or negative. Assigning 0% target weights to crypto.")
        total_top_n_volume = Decimal('0') # Prevent division by zero
    else:
         # Display volume with formatting
         print(f"Total 24hr Volume for Top {TOP_N_ASSETS_COUNT} crypto assets: {total_top_n_volume:,.2f} {QUOTE_ASSET}")

    # --- Calculate Dynamic Crypto Allocations ---
    # Percentage available for non-stablecoins
    crypto_allocation_pct = Decimal('100.0') - STABLECOIN_RESERVE_PCT

    if total_top_n_volume > 0:
        # Calculate weight based on volume contribution to the Top N total
        for index, row in top_n_crypto_df.iterrows():
            asset = row['Asset']
            volume = row[volume_col] # Should be Decimal

            # Calculate target percentage for this asset
            target_pct = (volume / total_top_n_volume) * crypto_allocation_pct
            dynamic_target_allocations_pct[asset] = target_pct
    else:
         # If total volume is zero, assign 0% target to all Top N crypto assets
         for asset in top_n_crypto_df['Asset'].tolist():
              dynamic_target_allocations_pct[asset] = Decimal('0.0')


    # --- Add Stablecoin Allocation ---
    # Assign the reserve percentage to the primary quote asset
    if QUOTE_ASSET in dynamic_target_allocations_pct:
         dynamic_target_allocations_pct[QUOTE_ASSET] += STABLECOIN_RESERVE_PCT
    else:
         dynamic_target_allocations_pct[QUOTE_ASSET] = STABLECOIN_RESERVE_PCT

    # --- Ensure all target assets (Top N + Stablecoins) are in the dict ---
    # Create a set of all assets that *should* have a target (even if 0%)
    all_intended_target_assets = set(top_n_crypto_df['Asset'].tolist() + STABLECOIN_ASSETS)
    for asset in all_intended_target_assets:
         if asset not in dynamic_target_allocations_pct:
              dynamic_target_allocations_pct[asset] = Decimal('0.0') # Assign 0% if missing

    # --- Verify Sum ---
    total_dynamic_pct = sum(dynamic_target_allocations_pct.values())
    # Display sum with formatting
    print(f"Target Allocations Sum: {total_dynamic_pct:.2f}%")
    # Use Decimal for comparison threshold
    if abs(total_dynamic_pct - Decimal('100.0')) > Decimal('0.01'):
        print("⚠️ WARNING: Target Sum is not 100%!")

    # --- Optional: Print sorted targets for review ---
    # print("\nCalculated Target Allocations (%):")
    # sorted_targets = dict(sorted(dynamic_target_allocations_pct.items(), key=lambda item: item[1], reverse=True))
    # for asset, pct in sorted_targets.items():
    #     if pct > 0: print(f"  {asset}: {pct:.2f}%") # Show non-zero targets with 2 decimals

except Exception as e:
    print(f"❌ An error occurred during dynamic target calculation: {e}")
    dynamic_target_allocations_pct = {} # Reset on error

if not dynamic_target_allocations_pct:
    print("⚠️ Dynamic target allocation dictionary is empty.")
else:
    print("✅ Dynamic target allocations calculated ('dynamic_target_allocations_pct').")


--- Calculating Dynamic Target Allocations ---
Total 24hr Volume for Top 10 crypto assets: 7,141,983.37 USDT
Target Allocations Sum: 100.00%
✅ Dynamic target allocations calculated ('dynamic_target_allocations_pct').


In [8]:
# Cell 8: Define Support/Resistance Helper Function

import numpy as np
import pandas as pd
from decimal import Decimal # Import Decimal

print("\n--- Defining Helper Function: find_simple_sr ---")

def find_simple_sr(df, lookback=20):
    """
    Identifies simple recent swing lows (support) and highs (resistance).
    Looks for the min Low and max High over a rolling lookback period.

    Args:
        df (pd.DataFrame): Kline DataFrame with 'Low' and 'High' columns.
                           Assumes these columns contain numeric types (float or Decimal).
        lookback (int): How many recent periods (rows) to consider.

    Returns:
        tuple: (recent_support, recent_resistance) as the original numeric type (float/Decimal)
               Returns (None, None) if not enough data or columns missing.
    """
    required_cols = ['Low', 'High']
    # Check if DataFrame is valid and has enough rows for the lookback
    if df is None or df.empty or len(df) < lookback or not all(c in df.columns for c in required_cols):
        return None, None # Not enough data or required columns missing

    try:
        # Get the relevant 'Low' and 'High' data for the lookback period
        # Use .iloc[-lookback:] for potentially better performance than .tail() on large dfs
        recent_data = df.iloc[-lookback:]
        recent_lows = recent_data['Low']
        recent_highs = recent_data['High']

        # Check if the slices contain valid (non-NaN) data before finding min/max
        if recent_lows.isnull().all() or recent_highs.isnull().all():
            # print(f"  Debug SR: All Lows or Highs are NaN in lookback period.") # Optional debug
            return None, None

        # Find the minimum low (support) and maximum high (resistance), skipping NaNs
        support = recent_lows.min(skipna=True)
        resistance = recent_highs.max(skipna=True)

        # Ensure we return None if min/max resulted in NaN (e.g., if all values were NaN despite previous check)
        if pd.isna(support) or pd.isna(resistance):
            return None, None

        # Return the values (will be float or Decimal depending on df input type)
        return support, resistance

    except Exception as e:
        print(f"  ❌ Error finding S/R: {e}")
        return None, None

print("✅ Helper function 'find_simple_sr' defined.")
# --- Example Test (Optional - Uncomment to run) ---
# if 'top_n_klines' in locals() and top_n_klines and 'top_n_symbols' in locals() and top_n_symbols:
#     test_symbol = top_n_symbols[0]
#     test_interval = '1d'
#     if test_symbol in top_n_klines and test_interval in top_n_klines[test_symbol]:
#         test_df = top_n_klines[test_symbol][test_interval]
#         print(f"\nRunning S/R example for {test_symbol} {test_interval}...")
#         # Ensure Low/High are numeric if needed (should be from processing)
#         # test_df['Low'] = pd.to_numeric(test_df['Low'], errors='coerce')
#         # test_df['High'] = pd.to_numeric(test_df['High'], errors='coerce')
#         sup, res = find_simple_sr(test_df, lookback=30) # Example lookback
#         print(f"Example {test_symbol} Daily S/R (lookback 30):")
#         if sup is not None and res is not None:
#             # Format output appropriately, checking if Decimal or float
#             sup_str = f"{sup:.8f}" if isinstance(sup, Decimal) else f"{sup:.4f}"
#             res_str = f"{res:.8f}" if isinstance(res, Decimal) else f"{res:.4f}"
#             print(f"  Support: {sup_str}, Resistance: {res_str}")
#             if 'Close' in test_df.columns:
#                 last_close = test_df['Close'].iloc[-1]
#                 close_str = f"{last_close:.8f}" if isinstance(last_close, Decimal) else f"{last_close:.4f}"
#                 print(f"  Last Close: {close_str}")
#                 # Perform subtraction carefully if types might differ
#                 try:
#                     dist_sup = Decimal(str(last_close)) - Decimal(str(sup))
#                     dist_res = Decimal(str(res)) - Decimal(str(last_close))
#                     print(f"  Dist to Support: {dist_sup:.8f}")
#                     print(f"  Dist to Resistance: {dist_res:.8f}")
#                 except Exception as calc_err:
#                     print(f"  Could not calculate distances: {calc_err}")
#         else:
#             print("  Could not calculate S/R (check data).")
#     else:
#         print(f"\nCannot run example S/R test: {test_symbol} {test_interval} data missing or invalid.")
# else:
#     print("\nCannot run example S/R test: Prerequisite data missing.")
# --- End Example Test ---


--- Defining Helper Function: find_simple_sr ---
✅ Helper function 'find_simple_sr' defined.


In [9]:
# Cell 9 (Restore): Asset Selection (Volume + Whitelist)

import pandas as pd
import os
from decimal import Decimal

# --- Configuration ---
N_VOLUME_CANDIDATES = 15 # How many top volume assets to consider initially
N_FINAL_TARGET = 10 # Aim for roughly this many assets in the final list
QUOTE_ASSET = 'USDT'
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD', 'TUSD', 'USDP', 'FDUSD']
MANUAL_WHITELIST_ASSETS = ['BTC', 'ETH'] # Always include these if available

# --- Check Prerequisites ---
if 'client' not in locals() or client is None: raise RuntimeError("Client not initialized.")

# --- Initialize ---
strategy_target_assets = [] # Use this name for final list
strategy_target_symbols = []
volume_ranking_df = pd.DataFrame() # Can be used later if needed

print("\n--- Selecting Assets (Top Volume + Whitelist) ---")
print(f"Whitelist: {MANUAL_WHITELIST_ASSETS}")

try:
    # 1. Fetch Tickers & Rank by Volume
    print(f"Fetching tickers & ranking top {N_VOLUME_CANDIDATES} by volume...")
    all_tickers = client.get_ticker()
    quote_pairs = [t for t in all_tickers if t.get('symbol','').endswith(QUOTE_ASSET)]
    volume_data = []; all_available_symbols = set()
    for ticker in quote_pairs:
        symbol = ticker.get('symbol');
        if not symbol: continue
        all_available_symbols.add(symbol)
        base_asset = symbol[:-len(QUOTE_ASSET)]
        if base_asset in STABLECOIN_ASSETS: continue
        try:
             volume = Decimal(ticker.get('quoteVolume','0.0')); price = Decimal(ticker.get('lastPrice','0.0'))
             if volume > 0 and price > 0: volume_data.append({'Asset': base_asset, 'Symbol': symbol, f'Volume_{QUOTE_ASSET}': volume})
        except Exception: pass
    if not volume_data: raise ValueError("No valid volume data found.")
    volume_ranking_df = pd.DataFrame(volume_data)
    volume_ranking_df[f'Volume_{QUOTE_ASSET}'] = volume_ranking_df[f'Volume_{QUOTE_ASSET}'].apply(Decimal)
    volume_ranking_df.sort_values(by=f'Volume_{QUOTE_ASSET}', ascending=False, inplace=True)

    # 2. Create Candidate Pool (Top N Vol + Whitelist)
    top_n_assets_set = set(volume_ranking_df.head(N_VOLUME_CANDIDATES)['Asset'].tolist())
    whitelisted_candidates = set()
    for asset in MANUAL_WHITELIST_ASSETS:
        symbol = f"{asset}{QUOTE_ASSET}"
        if symbol in all_available_symbols: whitelisted_candidates.add(asset)
        else: print(f"  Warn: Whitelisted {asset} not found.")
    candidate_assets_set = top_n_assets_set.union(whitelisted_candidates)

    # 3. Final Selection (Filter original DF, limit size, ensure whitelist)
    final_candidates_df = volume_ranking_df[volume_ranking_df['Asset'].isin(candidate_assets_set)].copy()
    if len(final_candidates_df) > N_FINAL_TARGET:
        final_candidates_df = final_candidates_df.head(N_FINAL_TARGET)
    strategy_target_assets = final_candidates_df['Asset'].tolist()
    for asset in MANUAL_WHITELIST_ASSETS: # Ensure whitelist included
         symbol = f"{asset}{QUOTE_ASSET}"
         if symbol in all_available_symbols and asset not in strategy_target_assets:
             print(f"  Re-adding required whitelisted asset: {asset}")
             asset_data = volume_ranking_df[volume_ranking_df['Asset'] == asset]
             if not asset_data.empty: strategy_target_assets.append(asset) ; # Only need asset name now
    # Deduplicate just in case
    strategy_target_assets = sorted(list(set(strategy_target_assets)), key = lambda x: volume_ranking_df[volume_ranking_df['Asset']==x].index[0] if x in volume_ranking_df['Asset'].values else float('inf')) # Sort approx by volume rank
    strategy_target_symbols = [f"{asset}{QUOTE_ASSET}" for asset in strategy_target_assets]

except Exception as e:
    print(f"❌ Error during asset selection: {e}"); strategy_target_assets, strategy_target_symbols = [], []

# --- Final Output ---
if not strategy_target_symbols: print("\n⚠️ Failed to determine target symbols.")
else:
    print(f"\n✅ Final Target Assets ({len(strategy_target_assets)}): {strategy_target_assets}")
    print(f"✅ Final Target Symbols: {strategy_target_symbols}")

if 'strategy_target_assets' not in locals(): strategy_target_assets=[]
if 'strategy_target_symbols' not in locals(): strategy_target_symbols=[]


--- Selecting Assets (Top Volume + Whitelist) ---
Whitelist: ['BTC', 'ETH']
Fetching tickers & ranking top 15 by volume...

✅ Final Target Assets (10): ['BTC', 'ETH', 'XRP', 'LTC', 'ADA', 'XLM', 'DOGE', 'SOL', 'HBAR', 'SUI']
✅ Final Target Symbols: ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LTCUSDT', 'ADAUSDT', 'XLMUSDT', 'DOGEUSDT', 'SOLUSDT', 'HBARUSDT', 'SUIUSDT']


In [10]:
# Cell 10: Calculate Target Allocations (Equal Weight)

import pandas as pd
import numpy as np
from decimal import Decimal

# --- Configuration ---
# Use the asset list generated in the previous cell (Cell 9)
ASSET_LIST_VARIABLE = 'strategy_target_assets'
# Define reserve and stablecoins
STABLECOIN_RESERVE_PCT = Decimal('20.0') # Fixed reserve for this strategy version
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD', 'TUSD', 'USDP', 'FDUSD']
QUOTE_ASSET = 'USDT' # Primary stablecoin

# --- Check Prerequisites ---
if ASSET_LIST_VARIABLE not in locals() or not locals()[ASSET_LIST_VARIABLE]:
     raise ValueError(f"List of target assets ('{ASSET_LIST_VARIABLE}') not found or empty. Run Cell 9 first.")

# --- Initialize ---
target_allocations_pct = {} # Stores final targets as Decimals
crypto_assets_to_allocate = locals()[ASSET_LIST_VARIABLE] # Get the list from Cell 9
num_crypto_assets = len(crypto_assets_to_allocate)

print("\n--- Calculating Target Allocations (Equal Weight) ---")
print(f"Targeting {num_crypto_assets} crypto assets: {crypto_assets_to_allocate}")
print(f"Stablecoin Reserve Target: {STABLECOIN_RESERVE_PCT}% ({QUOTE_ASSET})")

if num_crypto_assets <= 0:
    print("⚠️ No crypto assets selected. Assigning 100% to stablecoin.")
    target_allocations_pct[QUOTE_ASSET] = Decimal('100.0')
else:
    # Calculate Equal Weight for Crypto Assets
    crypto_total_pct = Decimal('100.0') - STABLECOIN_RESERVE_PCT
    # Use quantize for potentially cleaner division result if needed, though direct division is usually fine
    pct_per_crypto = (crypto_total_pct / Decimal(num_crypto_assets)).quantize(Decimal('0.0001')) # 4 decimal places for percentage
    print(f"Assigning ~{pct_per_crypto:.2f}% target to each crypto asset.")

    for asset in crypto_assets_to_allocate:
        target_allocations_pct[asset] = pct_per_crypto

    # Assign Stablecoin Reserve
    target_allocations_pct[QUOTE_ASSET] = STABLECOIN_RESERVE_PCT

# Ensure all stablecoins have a 0% target (unless primary)
for stable in STABLECOIN_ASSETS:
    if stable not in target_allocations_pct:
        target_allocations_pct[stable] = Decimal('0.0')

# Verify Sum & Adjust if necessary due to quantization/rounding
total_calculated_pct = sum(target_allocations_pct.values())
print(f"\nTarget Allocations Sum (Pre-Adjustment): {total_calculated_pct:.4f}%")
# Adjust the largest crypto allocation slightly if sum is off
if abs(total_calculated_pct - Decimal('100.0')) > Decimal('0.0001'):
    difference = Decimal('100.0') - total_calculated_pct
    # Find asset with largest allocation (excluding USDT) to adjust
    asset_to_adjust = max((asset for asset in crypto_assets_to_allocate),
                           key=lambda a: target_allocations_pct.get(a, Decimal(0)))
    if asset_to_adjust:
         target_allocations_pct[asset_to_adjust] += difference
         print(f"Adjusting {asset_to_adjust} target by {difference:.4f}% to reach 100% sum.")
         total_calculated_pct = sum(target_allocations_pct.values()) # Recalculate sum
         print(f"Target Allocations Sum (Post-Adjustment): {total_calculated_pct:.2f}%")
    else:
         print(f"⚠️ WARNING: Target Sum != 100% ({total_calculated_pct:.4f}%) and couldn't find asset to adjust.")

# Display Allocations
print("\n--- Final Target Allocations (%) ---")
sorted_targets = dict(sorted(target_allocations_pct.items(), key=lambda item: item[1], reverse=True))
for asset, pct in sorted_targets.items():
    if pct > 0: print(f"  {asset}: {pct:.2f}%") # Show non-zero targets

print("\n✅ Equal weight target allocations calculated.")
print("   Stored results in 'target_allocations_pct'.")

if 'target_allocations_pct' not in locals(): target_allocations_pct = {}


--- Calculating Target Allocations (Equal Weight) ---
Targeting 10 crypto assets: ['BTC', 'ETH', 'XRP', 'LTC', 'ADA', 'XLM', 'DOGE', 'SOL', 'HBAR', 'SUI']
Stablecoin Reserve Target: 20.0% (USDT)
Assigning ~8.00% target to each crypto asset.

Target Allocations Sum (Pre-Adjustment): 100.0000%

--- Final Target Allocations (%) ---
  USDT: 20.00%
  BTC: 8.00%
  ETH: 8.00%
  XRP: 8.00%
  LTC: 8.00%
  ADA: 8.00%
  XLM: 8.00%
  DOGE: 8.00%
  SOL: 8.00%
  HBAR: 8.00%
  SUI: 8.00%

✅ Equal weight target allocations calculated.
   Stored results in 'target_allocations_pct'.


In [11]:
# Cell 11 (Corrected Syntax v2): Fetch RSS News Feeds

# --- Necessary Imports ---
try: import feedparser
except ImportError: raise ImportError("feedparser not found. pip install feedparser")
try: import requests
except ImportError: raise ImportError("requests not found. pip install requests")
import pandas as pd
from datetime import datetime, timezone, timedelta
import time
import socket
import re

# --- Configuration ---
RSS_FEED_URLS = [ # Use relevant, working feeds
    "https://cointelegraph.com/rss/tag/bitcoin", "https://cointelegraph.com/rss/tag/ethereum",
    "https://www.coindesk.com/arc/outboundfeeds/rss/", "https://www.newsbtc.com/feed/",
    "https://cryptoslate.com/feed/", ]
MAX_ITEMS_PER_FEED = 5
MAX_ITEM_AGE_HOURS = 12
REQUEST_TIMEOUT = 10; SOCKET_TIMEOUT = 10

# --- Function Definition ---
print("\n--- Defining RSS Feed Fetching Function ---")
def fetch_recent_rss_entries(feed_urls, max_items=5, max_age_hours=24, req_timeout=10, sock_timeout=10):
    all_recent_entries = []; processed_links = set(); now_utc = datetime.now(timezone.utc)
    cutoff_time = now_utc - timedelta(hours=max_age_hours)
    print(f"Fetching RSS feeds (Max {max_items}/feed, <{max_age_hours} hrs old)...")
    original_timeout = socket.getdefaulttimeout(); socket.setdefaulttimeout(sock_timeout)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    for url in feed_urls:
        print(f"  Fetching: {url[:70]}...")
        entries_from_feed = []
        try:
            response = requests.get(url, headers=headers, timeout=req_timeout); response.raise_for_status()
            feed_data = feedparser.parse(response.text); count = 0
            if feed_data.bozo: print(f"    ⚠️ Feedparser issue: {getattr(feed_data, 'bozo_exception', 'Unknown')}")
            for entry in feed_data.entries:
                if count >= max_items: break
                link = getattr(entry, 'link', None)
                if not link or link in processed_links: continue
                published_time = None
                pub_parsed = getattr(entry, 'published_parsed', None) or getattr(entry, 'updated_parsed', None)
                # --- CORRECTED SYNTAX (Indentation) ---
                if pub_parsed:
                    try:
                        published_time = datetime.fromtimestamp(time.mktime(pub_parsed), timezone.utc)
                    except Exception:
                        pass # Ignore time parsing errors silently
                # --- END CORRECTION ---
                if published_time and published_time >= cutoff_time:
                    title = getattr(entry, 'title', 'N/A').strip()
                    summary = ' '.join(re.sub('<[^<]+?>', '', getattr(entry, 'summary', 'N/A')).split()).strip()
                    entries_from_feed.append({'title': title, 'link': link, 'summary': summary[:350] + ('...' if len(summary) > 350 else ''), 'published': published_time, 'source_feed': url})
                    processed_links.add(link); count += 1
            print(f"    -> Found {len(entries_from_feed)} recent items.")
            all_recent_entries.extend(entries_from_feed)
        except requests.exceptions.Timeout: print(f"    ❌ TIMEOUT: {url}")
        except requests.exceptions.RequestException as req_err: print(f"    ❌ REQ ERROR: {req_err}")
        except Exception as e: print(f"    ❌ UNEX ERROR: {e}")
        time.sleep(0.2)
    socket.setdefaulttimeout(original_timeout)
    all_recent_entries.sort(key=lambda x: x.get('published', datetime.min.replace(tzinfo=timezone.utc)), reverse=True)
    print(f"--- Finished fetching. Total recent: {len(all_recent_entries)} ---")
    return all_recent_entries
print("✅ Function 'fetch_recent_rss_entries' defined.")

# --- Execute the Function ---
print("\n--- Fetching Current News Items ---")
recent_news_items = fetch_recent_rss_entries(
    feed_urls=RSS_FEED_URLS, max_items=MAX_ITEMS_PER_FEED, max_age_hours=MAX_ITEM_AGE_HOURS,
    req_timeout=REQUEST_TIMEOUT, sock_timeout=SOCKET_TIMEOUT )

# --- Display ---
if not recent_news_items: print("\nNo recent relevant news items found.")
else: print(f"\n--- {len(recent_news_items)} Recent News Items (Sample) ---"); news_df = pd.DataFrame(recent_news_items); news_df['published_str'] = news_df['published'].dt.strftime('%Y-%m-%d %H:%M'); print(news_df[['published_str', 'title']].head(10))
print("\n✅ News fetching complete. Stored in 'recent_news_items'.")
if 'recent_news_items' not in locals(): recent_news_items = []


--- Defining RSS Feed Fetching Function ---
✅ Function 'fetch_recent_rss_entries' defined.

--- Fetching Current News Items ---
Fetching RSS feeds (Max 5/feed, <12 hrs old)...
  Fetching: https://cointelegraph.com/rss/tag/bitcoin...
    -> Found 5 recent items.
  Fetching: https://cointelegraph.com/rss/tag/ethereum...
    -> Found 2 recent items.
  Fetching: https://www.coindesk.com/arc/outboundfeeds/rss/...
    -> Found 5 recent items.
  Fetching: https://www.newsbtc.com/feed/...
    -> Found 5 recent items.
  Fetching: https://cryptoslate.com/feed/...
    -> Found 5 recent items.
--- Finished fetching. Total recent: 22 ---

--- 22 Recent News Items (Sample) ---
      published_str                                              title
0  2025-04-09 12:50  Bitcoin’s safe-haven appeal grows during trade...
1  2025-04-09 12:37  Binance's Second Reward-Bearing Asset LDUSDT t...
2  2025-04-09 12:17  Bitcoin emerges as major winner as China eyes ...
3  2025-04-09 12:00  XRP To Flip Ethereum: 

In [12]:
# Cell 12: Summarize News Context

import google.generativeai as genai
import pandas as pd
import json
from datetime import datetime, timezone
import time
import re
import os # Need os to check for env var existence robustly
from dotenv import load_dotenv # Ensure dotenv is loaded if needed

# --- Configuration ---\
LLM_MODEL_NAME_SUMMARIZER = "gemini-1.5-flash-latest"
MAX_NEWS_ITEMS_FOR_SUMMARY = 5 # Number of news items to feed into the summary prompt
MAX_CHARS_PER_ITEM_FOR_SUMMARY = 300 # Limit characters per news item in prompt

# --- Check Prerequisites ---\
news_summary = {
    "timestamp": datetime.now(timezone.utc).isoformat(),
    "summary": "Prerequisites not met or LLM skipped.",
    "sentiment": "Unknown",
    "themes": "N/A"
}
proceed_with_summary = False

# Ensure genai_configured flag exists from Cell 3
if 'genai_configured' not in locals():
    # Attempt to re-check configuration if flag is missing (e.g., cell run out of order)
    print("⚠️ 'genai_configured' flag not found, re-checking LLM configuration...")
    genai_configured = False # Default
    try:
        if 'genai' in locals() and genai is not None: # Check if genai object exists
             load_dotenv()
             google_api_key = os.environ.get('GOOGLE_API_KEY')
             if google_api_key:
                 genai.configure(api_key=google_api_key)
                 genai_configured = True # Set flag on success
                 print("✅ LLM re-configured.")
             else:
                 print("⚠️ GOOGLE_API_KEY not found on re-check.")
        else:
             print("⚠️ google.generativeai object not found on re-check.")
    except Exception as e:
        print(f"❌ Error re-configuring Google AI: {e}")


# Now perform the checks using the potentially updated genai_configured flag
if not genai_configured:
    print("⚠️ LLM not configured. Skipping news summarization.")
elif 'recent_news_items' not in locals():
    print("⚠️ 'recent_news_items' list not found (Run Cell 11?).")
    news_summary['summary'] = "News fetch failed or variable missing."
elif not recent_news_items:
    print("ℹ️ No news items found in 'recent_news_items' (Cell 11 result was empty).")
    news_summary['summary'] = "No recent news available to summarize."
else:
    proceed_with_summary = True

# --- Main Summarization Logic ---\
if proceed_with_summary:
    print(f"\n--- Summarizing Recent News Context ({LLM_MODEL_NAME_SUMMARIZER}) ---")
    # Prepare News Text for Prompt
    news_text_for_prompt = ""
    item_count = 0
    char_count = 0
    print(f"Preparing summary prompt using up to {MAX_NEWS_ITEMS_FOR_SUMMARY} news items...")

    # Iterate through the fetched news items (newest first, assuming sorted)
    for item in recent_news_items:
        if item_count >= MAX_NEWS_ITEMS_FOR_SUMMARY:
            break

        title = item.get('title', 'N/A')
        summary = item.get('summary', 'N/A')
        published_dt = item.get('published')
        published_str = published_dt.strftime('%Y-%m-%d %H:%M') if published_dt else "Unknown Time"

        # Create text for this item, truncate if necessary
        item_text_full = f"[{published_str}] {title}: {summary}"
        item_text_truncated = item_text_full[:MAX_CHARS_PER_ITEM_FOR_SUMMARY]
        if len(item_text_full) > MAX_CHARS_PER_ITEM_FOR_SUMMARY:
            item_text_truncated += "..."

        # Basic check to avoid overly large prompts (adjust limit as needed)
        if char_count + len(item_text_truncated) > 8000: # Safety break
             print("  Warning: Approaching character limit for prompt, stopping news inclusion.")
             break

        news_text_for_prompt += f"- {item_text_truncated}\n"
        char_count += len(item_text_truncated) + 3 # +3 for "- " and newline
        item_count += 1

    print(f"Prepared {item_count} news items for prompt.")

    # Simplified Prompt focusing ONLY on summarization
    summarization_prompt = f"""
    Please provide a concise summary (2-3 sentences) of the following recent cryptocurrency news items. Focus on the main events or sentiments expressed.

    NEWS ITEMS:
    {news_text_for_prompt}

    OUTPUT: Just the summary text. No greetings, no extra commentary.
    """

    # Call LLM
    try:
        model = genai.GenerativeModel(LLM_MODEL_NAME_SUMMARIZER)
        # These safety settings are needed to ALLOW summarization, which Google
        # might sometimes flag as "dangerous" if news mentions finance/risk.
        # Use with caution and understand the model might still refuse.
        safety_settings=[
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]

        print("Sending simplified request to LLM for news summary...")
        # Adding generation_config for potentially better control if needed later
        # generation_config = genai.types.GenerationConfig(
        #     # candidate_count=1, # Default is 1
        #     # stop_sequences=['\n\n'], # Example if needed
        #     # max_output_tokens=150, # Example limit
        #     temperature=0.7 # Adjust creativity vs consistency
        # )
        response = model.generate_content(
            summarization_prompt,
            safety_settings=safety_settings,
            # generation_config=generation_config # Uncomment to use config
            )

        # Check for blocks or empty response
        if not response.candidates or (response.prompt_feedback and response.prompt_feedback.block_reason):
            reason = response.prompt_feedback.block_reason if response.prompt_feedback else "Unknown reason (no candidates)"
            print(f"❌ LLM Response BLOCKED or Empty. Reason: {reason}")
            news_summary['summary'] = f"LLM Response Blocked/Empty: {reason}"
            news_summary['sentiment'] = "Blocked/Error"
            news_summary['themes'] = "Blocked/Error"
        else:
            # Successfully received response
            raw_summary_text = response.text.strip()
            print(f"\n--- LLM Raw Summary Output ---\n{raw_summary_text}\n--- End Raw Summary ---")

            # Store the summary, mark others as not requested for this simplified call
            news_summary['summary'] = raw_summary_text
            news_summary['sentiment'] = "Unknown (Not Requested)"
            news_summary['themes'] = "Unknown (Not Requested)"
            news_summary['timestamp'] = datetime.now(timezone.utc).isoformat() # Update timestamp
            print("\n✅ News context summarized successfully (simplified request).")

    except Exception as e:
        print(f"❌ Error during LLM news summarization: {e}")
        news_summary['summary'] = f"LLM API Error: {str(e)}"
        news_summary['sentiment'] = "Error"
        news_summary['themes'] = "Error"
else:
    # This handles cases where prerequisites weren't met
    print("\nSkipped news summarization due to missing prerequisites or empty news list.")

# --- Final Output ---\
print("\n--- Current News Summary ---")
print(f"News Sentiment: {news_summary.get('sentiment', 'Unknown')}")
print(f"News Themes: {news_summary.get('themes', 'N/A')}")
print(f"News Summary: {news_summary.get('summary', 'N/A')}")
print(f"(Generated: {news_summary.get('timestamp')})")

print("\nNews summary stored in 'news_summary' dictionary.")

# Ensure news_summary variable exists in the global scope even if skipped
if 'news_summary' not in locals():
    news_summary = {"summary": "Variable not created.", "sentiment": "Unknown"}

# Reminder about MACRO_CONTEXT_SUMMARY if it exists from previous runs/manual definitions
# (This variable is no longer actively used in the core summarization prompt)
if 'MACRO_CONTEXT_SUMMARY' in locals():
     print("\nWarning: 'MACRO_CONTEXT_SUMMARY' variable still exists in the environment. It's not used by this cell but might be present from older versions.")
     # Consider removing it if definitively unused:
     # del MACRO_CONTEXT_SUMMARY


--- Summarizing Recent News Context (gemini-1.5-flash-latest) ---
Preparing summary prompt using up to 5 news items...
Prepared 5 news items for prompt.
Sending simplified request to LLM for news summary...

--- LLM Raw Summary Output ---
Increased global trade tensions are boosting Bitcoin's status as a safe-haven asset.  Binance is launching a second reward-bearing asset, LDUSDT.  Meanwhile, predictions for XRP's market dominance and concerns over rising UK bond yields add to the complex cryptocurrency market landscape.
--- End Raw Summary ---

✅ News context summarized successfully (simplified request).

--- Current News Summary ---
News Sentiment: Unknown (Not Requested)
News Themes: Unknown (Not Requested)
News Summary: Increased global trade tensions are boosting Bitcoin's status as a safe-haven asset.  Binance is launching a second reward-bearing asset, LDUSDT.  Meanwhile, predictions for XRP's market dominance and concerns over rising UK bond yields add to the complex cryptocu

In [13]:
# Cell 13: Fetch/Load Historical Data for Selected Assets

import pandas as pd
import os
from binance.client import Client # For interval constants
from datetime import datetime, timezone # For timezone handling
import time # Potentially useful for slight delays if needed

# --- Configuration ---\
# Define intervals needed for TA and potentially other logic
INTERVALS_TO_PROCESS = ['1d', '1h', '5m', '1m']
# Map user-friendly keys to Binance API constants
INTERVAL_API_MAP = {
    '1d': Client.KLINE_INTERVAL_1DAY,
    '1h': Client.KLINE_INTERVAL_1HOUR,
    '5m': Client.KLINE_INTERVAL_5MINUTE,
    '1m': Client.KLINE_INTERVAL_1MINUTE
}
# Define fetch ranges (used if fetching fresh data)
START_DATES_FETCH = {
    '1d': "90 days ago UTC",
    '1h': "7 days ago UTC",
    '5m': "3 days ago UTC",
    '1m': "1 day ago UTC"
}
# Define identifiers used in filenames when saving/loading CSV data
START_DATES_SAVE_IDS = {
    '1d': "90_days_ago_UTC",
    '1h': "7_days_ago_UTC",
    '5m': "3_days_ago_UTC",
    '1m': "1_day_ago_UTC"
}
DATA_DIRECTORY = "data" # Subdirectory to store/load CSV files
FORCE_FETCH_FRESH = False # Set True to always ignore local CSVs and fetch from API

# --- Check Prerequisites ---\
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client is not initialized. Run Cell 2 first.")
# Use the asset list generated in Cell 9
if 'strategy_target_symbols' not in locals() or not strategy_target_symbols:
    raise ValueError("'strategy_target_symbols' list not found or empty. Run Cell 9 first.")
# Ensure fetch helper function exists (should be from Cell 1)
if 'fetch_and_process_klines' not in locals():
     raise RuntimeError("Helper function 'fetch_and_process_klines' not defined. Run Cell 1.")

# --- Initialization ---\
selected_asset_klines = {} # Dictionary structure: {'SYMBOL': {'interval': DataFrame}}
os.makedirs(DATA_DIRECTORY, exist_ok=True) # Ensure data directory exists
errors = {'load': 0, 'fetch': 0, 'save': 0} # Track errors by type
missing_symbols_data = [] # Track symbols missing some intervals

print(f"\n--- Processing Kline Data for {len(strategy_target_symbols)} Selected Symbols ---")
print(f"Symbols: {strategy_target_symbols}")
print(f"Intervals: {INTERVALS_TO_PROCESS}")
print(f"Will {'FETCH FRESH' if FORCE_FETCH_FRESH else 'LOAD local first'}.")

# Loop through each symbol selected in Cell 9
for symbol in strategy_target_symbols:
    selected_asset_klines[symbol] = {} # Initialize nested dictionary for the symbol
    symbol_fetch_errors = 0
    symbol_intervals_loaded = 0

    # Loop through each required interval for the current symbol
    for interval_key in INTERVALS_TO_PROCESS:
        interval_value = INTERVAL_API_MAP.get(interval_key)
        if not interval_value:
             print(f"  ⚠️ Skipping invalid interval key: {interval_key} for {symbol}")
             continue # Skip if interval key isn't mapped

        df = None # Reset DataFrame for each interval
        loaded_from_csv = False
        source_msg = "" # To track if loaded or fetched

        # --- 1. Attempt to Load from CSV ---\
        if not FORCE_FETCH_FRESH:
            start_id = START_DATES_SAVE_IDS.get(interval_key)
            if start_id:
                filename = os.path.join(DATA_DIRECTORY, f"{symbol}_{interval_key}_{start_id}.csv")
                if os.path.exists(filename):
                    try:
                        # Read CSV, set index, parse dates
                        df = pd.read_csv(filename, index_col='Open Time', parse_dates=True)

                        # Consistent Timezone Handling (Crucial!)
                        if df.index.tz is None:
                            # If no timezone info, assume it should be UTC and localize
                            df.index = df.index.tz_localize('UTC')
                            # print(f"  Localized {filename} index to UTC.") # Optional debug
                        elif df.index.tz != timezone.utc:
                            # If it has a timezone but it's not UTC, convert it
                             print(f"  Warning: Timezone for {filename} is {df.index.tz}, converting to UTC.")
                             df.index = df.index.tz_convert('UTC')

                        # Validate DF after loading and timezone adjustment
                        if df is not None and not df.empty:
                             loaded_from_csv = True
                             source_msg = f"Loaded {symbol} {interval_key} from CSV."
                             # print(f"  {source_msg}") # Optional: Confirm load success
                        else:
                             df = None # Ensure df is None if loading resulted in empty df

                    except Exception as e:
                        print(f"  ⚠️ Error loading {filename}: {e}. Will try fetching.")
                        errors['load'] += 1
                        df = None # Ensure df is None if loading fails

        # --- 2. Fetch Fresh if Not Loaded or if Forcing ---\
        if df is None: # Fetch if df is None (due to not existing, load error, or FORCE_FETCH_FRESH)
            start_str = START_DATES_FETCH.get(interval_key, "7 days ago UTC") # Default fallback
            # Fetch message handled within the helper function now
            # print(f"  Fetching {symbol} {interval_key} data starting {start_str}...")
            # Call the helper function defined in Cell 1
            df = fetch_and_process_klines(client, symbol, interval_value, start_str)
            source_msg = f"Fetched {symbol} {interval_key} from API."

            # --- 3. Optionally Save Freshly Fetched Data ---\
            if df is not None and not df.empty:
                 save_id = START_DATES_SAVE_IDS.get(interval_key)
                 if save_id:
                      save_filename = os.path.join(DATA_DIRECTORY, f"{symbol}_{interval_key}_{save_id}.csv")
                      try:
                           df.to_csv(save_filename)
                           # print(f"  Saved {save_filename}") # Optional: Confirm save success
                      except Exception as e:
                           print(f"  ⚠️ Error SAVING {save_filename}: {e}")
                           errors['save'] += 1
                 else:
                      print(f"  ⚠️ Cannot determine save filename for {interval_key}. Data not saved.")
            elif df is None: # Indicates an error during fetch/process reported by helper
                 symbol_fetch_errors += 1
                 # Error message printed by helper function

        # --- 4. Store the final DataFrame ---\
        if df is not None and not df.empty:
             selected_asset_klines[symbol][interval_key] = df
             symbol_intervals_loaded += 1
        elif not loaded_from_csv: # Only flag error if fetch failed (load failure already logged)
             print(f"  ⚠️ No data obtained or processed for {symbol} {interval_key}.")

    # --- End of interval loop ---
    if symbol_intervals_loaded < len(INTERVALS_TO_PROCESS):
         missing_intervals = set(INTERVALS_TO_PROCESS) - set(selected_asset_klines.get(symbol, {}).keys())
         print(f"  -> Note: {symbol} processed, but missing interval(s): {', '.join(missing_intervals)}")
         if symbol not in missing_symbols_data:
             missing_symbols_data.append(symbol)

    if symbol_fetch_errors > 0:
        errors['fetch'] += symbol_fetch_errors
        # Detailed fetch errors already printed by helper

# --- End of symbol loop ---\

# --- Final Summary ---\
print(f"\nFinished processing data.")
for error_type, count in errors.items():
    if count > 0:
        print(f"⚠️ Encountered {count} {error_type.capitalize()} error(s) during processing.")

if missing_symbols_data:
     print(f"⚠️ Symbols potentially missing required interval data: {list(set(missing_symbols_data))}")

KLINE_DICT_NAME = 'selected_asset_klines' # Define the output variable name explicitly
print(f"\n✅ Kline data stored in '{KLINE_DICT_NAME}'.")

# Final check to ensure the dictionary exists and report if empty
if KLINE_DICT_NAME not in locals():
     locals()[KLINE_DICT_NAME] = {} # Create empty dict if it somehow doesn't exist
     print(f"⚠️ '{KLINE_DICT_NAME}' was not created during processing!")
elif not locals()[KLINE_DICT_NAME]:
     print(f"⚠️ '{KLINE_DICT_NAME}' is empty! No data was loaded or fetched successfully.")
elif len(locals()[KLINE_DICT_NAME]) != len(strategy_target_symbols):
     print(f"⚠️ Warning: Processed {len(locals()[KLINE_DICT_NAME])} symbols, expected {len(strategy_target_symbols)}.")


--- Processing Kline Data for 10 Selected Symbols ---
Symbols: ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LTCUSDT', 'ADAUSDT', 'XLMUSDT', 'DOGEUSDT', 'SOLUSDT', 'HBARUSDT', 'SUIUSDT']
Intervals: ['1d', '1h', '5m', '1m']
Will LOAD local first.

Finished processing data.

✅ Kline data stored in 'selected_asset_klines'.


In [14]:
# Cell 14: Calculate Indicators for Selected Assets

import pandas as pd
import numpy as np # Needed for manual calculations and NaN handling

# --- Configuration ---\
# Match periods used in indicator calculations (should align with Cell 6 if consistency is desired)
SMA_PERIODS = [20, 50]      # Periods for Simple Moving Averages
RSI_PERIOD = 14             # Period for Relative Strength Index
MACD_FAST = 12              # Fast EMA period for MACD
MACD_SLOW = 26              # Slow EMA period for MACD
MACD_SIGNAL = 9             # Signal Line EMA period for MACD

# --- Check Prerequisites ---\
KLINE_DATA_DICT = 'selected_asset_klines' # Input variable name from Cell 13
if KLINE_DATA_DICT not in locals() or not locals()[KLINE_DATA_DICT]:
    raise ValueError(f"Kline data dictionary ('{KLINE_DATA_DICT}') not found or empty. Run Cell 13 first.")

print("\n--- Calculating Technical Indicators for Selected Assets ---")
calculation_errors = 0
# Get the dictionary from the local scope
kline_data = locals()[KLINE_DATA_DICT]

# Loop through each symbol in the main dictionary
for symbol, interval_dict in kline_data.items():
    # Check if the symbol has any interval data loaded
    if not interval_dict:
        # print(f"  Skipping {symbol}: No interval data found.") # Optional debug
        continue

    symbol_errors = 0
    # Loop through each interval ('1d', '1h', etc.) for the current symbol
    for interval, df in interval_dict.items():
        # Ensure DataFrame is not empty and has the 'Close' column needed
        if df.empty or 'Close' not in df.columns:
            # print(f"  Skipping {symbol} {interval}: DataFrame empty or missing 'Close' column.") # Optional debug
            continue

        # --- Perform Calculations within a try-except block per DataFrame ---
        try:
            # --- Calculate SMAs ---
            for period in SMA_PERIODS:
                sma_col_name = f'SMA_{period}'
                # Ensure enough data points for the rolling window
                if len(df) >= period:
                    # Use min_periods=period to avoid partial calculations at the start
                    df[sma_col_name] = df['Close'].rolling(window=period, min_periods=period).mean()
                else:
                    df[sma_col_name] = np.nan # Assign NaN if not enough data

            # --- Calculate RSI ---
            rsi_col_name = f'RSI_{RSI_PERIOD}'
            # Need at least period+1 rows for diff() to work properly
            if len(df) >= RSI_PERIOD + 1:
                delta = df['Close'].diff()
                gain = delta.where(delta > 0, 0.0) # Get gains, default to 0
                loss = -delta.where(delta < 0, 0.0) # Get losses (as positive values), default to 0

                # Calculate Average Gain/Loss using Exponential Moving Average (common for RSI)
                avg_gain = gain.ewm(com=RSI_PERIOD - 1, min_periods=RSI_PERIOD).mean()
                avg_loss = loss.ewm(com=RSI_PERIOD - 1, min_periods=RSI_PERIOD).mean()

                # Calculate Relative Strength (RS) - handle division by zero
                # Where avg_loss is 0, RS is theoretically infinite (set RSI to 100 later)
                rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss)

                # Calculate RSI
                rsi = 100.0 - (100.0 / (1.0 + rs))
                # Handle the infinite RS case explicitly
                rsi[rs == np.inf] = 100.0

                # Ensure it's a Series with the correct index before filling NaNs
                rsi_series = pd.Series(rsi, index=df.index)

                # --- CORRECTED fillna ---
                # Backfill initial NaNs from EWM calculation (up to RSI period length)
                rsi_series.bfill(limit=RSI_PERIOD, inplace=True) # Use bfill() directly
                # Fill any remaining NaNs (usually only at the very start if bfill didn't reach) with 50
                rsi_series.fillna(50.0, inplace=True)
                # --- END CORRECTION ---

                df[rsi_col_name] = rsi_series
            else:
                df[rsi_col_name] = np.nan # Assign NaN if not enough data

            # --- Calculate MACD ---
            macd_col = f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'
            macds_col = f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}' # Signal line
            macdh_col = f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}' # Histogram

            # Need enough data for the slow EMA calculation
            if len(df) >= MACD_SLOW:
                ema_fast = df['Close'].ewm(span=MACD_FAST, adjust=False).mean()
                ema_slow = df['Close'].ewm(span=MACD_SLOW, adjust=False).mean()
                macd_line = ema_fast - ema_slow

                # Need enough *valid* MACD line points for the signal line EMA
                # Check non-NaN count of macd_line before calculating signal
                if macd_line.dropna().shape[0] >= MACD_SIGNAL:
                     signal_line = macd_line.ewm(span=MACD_SIGNAL, adjust=False).mean()
                     histogram = macd_line - signal_line
                else:
                     # Not enough data to calculate signal line reliably
                     signal_line = np.nan
                     histogram = np.nan

                df[macd_col] = macd_line
                df[macds_col] = signal_line
                df[macdh_col] = histogram
            else:
                 # Assign NaN if not enough data for initial MACD calculation
                 df[macd_col] = np.nan
                 df[macds_col] = np.nan
                 df[macdh_col] = np.nan

            # Note: DataFrames are modified in-place within the kline_data dictionary

        except Exception as e:
            print(f"  ❌ Error calculating indicators for {symbol} {interval}: {e}")
            symbol_errors += 1
            # Attempt cleanup of potentially partially added columns on error
            potential_new_cols = [f'SMA_{p}' for p in SMA_PERIODS] + \
                                 [f'RSI_{RSI_PERIOD}', macd_col, macds_col, macdh_col]
            for col in potential_new_cols:
                 if col in df.columns:
                      try:
                          df.drop(columns=[col], inplace=True)
                      except Exception:
                          pass # Ignore cleanup errors

    # Accumulate errors from this symbol
    if symbol_errors > 0:
        calculation_errors += symbol_errors

# --- Final Summary ---
if calculation_errors > 0:
     print(f"\n⚠️ Indicator calculation completed with {calculation_errors} total errors across all symbols/intervals.")

# --- Optional: Display sample data to verify one symbol/interval ---
try:
     # Check if strategy_target_symbols exists and is not empty
     if 'strategy_target_symbols' in locals() and strategy_target_symbols:
         symbol_to_show = strategy_target_symbols[0] # Show first selected symbol
         interval_to_show = '1h' # Choose an interval to display
         if symbol_to_show in kline_data and interval_to_show in kline_data[symbol_to_show]:
             print(f"\n--- Sample {symbol_to_show} {interval_to_show} Data with Indicators (Last 5 Rows) ---")
             df_sample = kline_data[symbol_to_show][interval_to_show]
             # Define columns to show (ensure they exist after calculation)
             cols_to_display = ['Close', f'SMA_{SMA_PERIODS[0]}', f'SMA_{SMA_PERIODS[1]}',
                                f'RSI_{RSI_PERIOD}', f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}']
             # Filter list to only include columns that actually exist in the DataFrame
             cols_to_display = [c for c in cols_to_display if c in df_sample.columns]
             if cols_to_display:
                 with pd.option_context('display.float_format', '{:.4f}'.format, 'display.max_rows', 5):
                     print(df_sample[cols_to_display].tail())
             else:
                 print("  Could not find expected indicator columns in sample DataFrame.")
         else:
             print(f"\nSample display skipped: Data for {symbol_to_show} {interval_to_show} not found.")
     else:
         print("\nSample display skipped: 'strategy_target_symbols' list is empty or missing.")
except Exception as display_err:
     print(f"\nError during sample display: {display_err}")
# --- End Optional Display ---

print(f"\n✅ Indicator calculation complete for data in '{KLINE_DATA_DICT}'.")


--- Calculating Technical Indicators for Selected Assets ---

--- Sample BTCUSDT 1h Data with Indicators (Last 5 Rows) ---
                               Close     SMA_20     SMA_50  RSI_14  \
Open Time                                                            
2025-04-07 14:00:00+00:00 79000.0900 77598.5935 80705.8584 52.6313   
2025-04-07 15:00:00+00:00 78790.5400 77557.4935 80628.3876 51.2791   
2025-04-07 16:00:00+00:00 77362.1600 77486.2510 80515.9292 43.1424   
2025-04-07 17:00:00+00:00 79051.6000 77491.2750 80445.6094 52.7019   
2025-04-07 18:00:00+00:00 78835.3000 77536.5630 80369.3856 51.5079   

                           MACD_12_26_9  
Open Time                                
2025-04-07 14:00:00+00:00     -936.0072  
2025-04-07 15:00:00+00:00     -769.5557  
2025-04-07 16:00:00+00:00     -744.3199  
2025-04-07 17:00:00+00:00     -581.2959  
2025-04-07 18:00:00+00:00     -464.2007  

✅ Indicator calculation complete for data in 'selected_asset_klines'.


In [15]:
# Cell 15: Filter Ideal Trades by Updated Feasibility

import pandas as pd
import numpy as np
from decimal import Decimal

# --- Configuration ---
QUOTE_ASSET = 'USDT'

# --- Check Prerequisites ---
# Use ideal trades from Cell 11
if 'trades_df' not in locals() or trades_df.empty:
     print("\n'trades_df' (ideal trades) is empty/missing. No trades to filter.")
     final_trades_to_format_df = pd.DataFrame()
# Use UPDATED balances from Cell 14
elif 'updated_balances_df' not in locals():
     print("⚠️ Updated balances ('updated_balances_df') not found. Cannot check final feasibility.")
     final_trades_to_format_df = pd.DataFrame() # Cannot proceed
else:
    print("\n--- Filtering Ideal Trades by Updated Feasibility ---")

    # Copy ideal trades to start filtering
    final_trades_to_format_df = trades_df.copy()
    final_trades_to_format_df['Feasible_Updated'] = True # Add new feasibility column
    final_trades_to_format_df['Feasibility_Reason'] = ''

    balances = updated_balances_df # Use the LATEST balances

    # Get LATEST free USDT
    free_usdt = balances.loc[QUOTE_ASSET, 'Free'] if QUOTE_ASSET in balances.index else Decimal('0.0')
    print(f"Available Free {QUOTE_ASSET} (Updated): {free_usdt:.8f}")

    cumulative_buy_cost = Decimal('0.0') # Track cost for updated check

    # Iterate through the IDEAL trades
    for index, trade in final_trades_to_format_df.iterrows():
        asset = trade['Asset']; action = trade['Action']
        ideal_qty = Decimal(str(trade['IdealQuantity']))
        ideal_value = Decimal(str(trade['IdealValueUSDT']))

        # Check SELL Feasibility vs UPDATED Free balance
        if action == 'SELL':
            free_balance = balances.loc[asset,'Free'] if asset in balances.index else Decimal('0.0')
            if ideal_qty > free_balance:
                final_trades_to_format_df.loc[index, 'Feasible_Updated'] = False
                final_trades_to_format_df.loc[index, 'Feasibility_Reason'] = f'Insufficient Free {asset} ({free_balance:.8f})'

        # Check BUY Feasibility vs UPDATED Free USDT (Cumulative)
        elif action == 'BUY':
            required_usdt = ideal_value
            if cumulative_buy_cost + required_usdt > free_usdt:
                final_trades_to_format_df.loc[index, 'Feasible_Updated'] = False
                final_trades_to_format_df.loc[index, 'Feasibility_Reason'] = f'Insufficient Free {QUOTE_ASSET} (Need {required_usdt:.2f}, Avail {(free_usdt - cumulative_buy_cost):.2f})'
            else:
                # Only increment cumulative cost if feasible based on UPDATED balance
                cumulative_buy_cost += required_usdt

    # Filter DataFrame based on the new Feasible_Updated column
    final_trades_to_format_df = final_trades_to_format_df[final_trades_to_format_df['Feasible_Updated']].copy()

    # --- Report Summary ---
    proceed_count = len(final_trades_to_format_df)
    original_count = len(trades_df) # Count from Cell 11's output DF
    skipped_count = original_count - proceed_count

    print(f"\nFinal Feasibility Check Summary (using updated balances):")
    print(f"  {proceed_count} / {original_count} ideal trades are feasible.")
    if skipped_count > 0:
        print(f"  {skipped_count} ideal trades skipped due to updated balance constraints.")
        # Optional: Display skipped trades and reasons if needed

    if not final_trades_to_format_df.empty:
        print("\n--- Trades Ready for Formatting ---")
        display_cols = ['Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT']
        def fmt(x,p=8): return f"{x:.{p}f}" if isinstance(x,Decimal) else (f"{x:.{p}f}" if isinstance(x,(int,float)) else str(x))
        formatters = {'IdealQuantity':lambda x:fmt(x,8), 'IdealValueUSDT':lambda x:fmt(x,2)}
        print(final_trades_to_format_df[display_cols].to_string(formatters=formatters, index=False))
        print("\n✅ Filtering complete. Feasible trades stored in 'final_trades_to_format_df'.")
    else:
        print("\n✅ Filtering complete. No ideal trades were feasible with updated balances.")


# Ensure df exists
if 'final_trades_to_format_df' not in locals(): final_trades_to_format_df = pd.DataFrame()


'trades_df' (ideal trades) is empty/missing. No trades to filter.


In [16]:
# Cell 16: Calculate Ideal Rebalancing Trades (Deviation-Based)
# NOTE: Cell number in comment may differ from execution order if cells were inserted/deleted.
# This cell calculates trades based on portfolio deviation from target allocations.

import pandas as pd
import numpy as np
from decimal import Decimal, InvalidOperation

# --- Configuration ---\
QUOTE_ASSET = 'USDT'
MIN_TRADE_VALUE_USDT = Decimal('10.0') # Minimum value for a trade to be considered significant
DISPLAY_PRECISION_QTY = 8
DISPLAY_PRECISION_VAL = 2
# Use a recent, frequently updated interval for price calculation
# '5m' or '1m' are often preferred for current valuation if data is reliable.
# '1h' is a safer fallback if shorter intervals are patchy.
PRICE_SOURCE_INTERVAL = '5m'

# --- Check Prerequisites ---\
if 'portfolio_state' not in locals() or not isinstance(portfolio_state, dict) or 'balances' not in portfolio_state:
    raise ValueError("Initial 'portfolio_state' (with 'balances' DataFrame) not found. Run Cell 2.")
if 'target_allocations_pct' not in locals() or not isinstance(target_allocations_pct, dict) or not target_allocations_pct:
    raise ValueError("Target allocations ('target_allocations_pct') not found or empty. Run Cell 10.")
# Use the kline data with indicators from Cell 14 for latest prices
if 'selected_asset_klines' not in locals() or not isinstance(selected_asset_klines, dict) or not selected_asset_klines:
     raise ValueError("Kline data ('selected_asset_klines') not found or empty. Run Cell 13 & 14.")
# Check if client exists for potential ticker fallback
if 'client' not in locals() or client is None:
     print("⚠️ Binance client not found. Ticker price fallback will not be available.")
     # Allow to continue, but price fetching might fail if klines are missing

# --- Initialize ---\
trades_df = pd.DataFrame(columns=[
    'Asset', 'Action', 'IdealQuantity', 'IdealValueUSDT', 'CurrentValueUSDT',
    'TargetValueUSDT', 'DeviationValueUSDT', 'DeviationPercent', 'CurrentPct',
    'TargetPct', 'CalcPrice' # Add CalcPrice column
])
calculation_success = False # Flag to track success

print("\n--- Determining Ideal Rebalancing Trades (Based on Total Value Deviation) ---")

try:
    # --- 1. Calculate Current Portfolio Value ---
    print("Calculating current portfolio value...")
    current_portfolio_value = Decimal('0.0')
    current_allocations_value = {} # Store value of each asset
    latest_prices = {} # Store prices used for calculation

    balances_df = portfolio_state['balances'] # Use initial balances from Cell 2

    # Get USDT balance first
    if QUOTE_ASSET in balances_df.index:
        usdt_total = balances_df.loc[QUOTE_ASSET, 'Free'] + balances_df.loc[QUOTE_ASSET, 'Locked']
        current_portfolio_value += usdt_total
        current_allocations_value[QUOTE_ASSET] = usdt_total
        latest_prices[QUOTE_ASSET] = Decimal('1.0') # Price of USDT is 1
        print(f"  {QUOTE_ASSET} Balance: {usdt_total:.{DISPLAY_PRECISION_VAL}f}")
    else:
         # Ensure USDT is included even if balance is zero, for target comparison
         current_allocations_value[QUOTE_ASSET] = Decimal('0.0')
         latest_prices[QUOTE_ASSET] = Decimal('1.0')


    # Iterate through assets in our balance sheet
    for asset in balances_df.index:
        if asset == QUOTE_ASSET:
            continue # Already handled

        symbol = f"{asset}{QUOTE_ASSET}"
        total_balance = balances_df.loc[asset, 'Free'] + balances_df.loc[asset, 'Locked']

        if total_balance <= Decimal('0.0'): # Skip if no balance, ensure it's in dict with 0 value
            current_allocations_value[asset] = Decimal('0.0')
            latest_prices[asset] = None # Mark price as unknown initially
            continue

        # Find latest price from kline data (use specified interval)
        price = None
        if symbol in selected_asset_klines and PRICE_SOURCE_INTERVAL in selected_asset_klines[symbol]:
            kline_df = selected_asset_klines[symbol][PRICE_SOURCE_INTERVAL]
            # Check if DF is not empty and has the 'Close' column
            if not kline_df.empty and 'Close' in kline_df.columns:
                try:
                    # Get the last non-NaN close price from the specified interval
                    last_valid_price = kline_df['Close'].dropna().iloc[-1]
                    price = Decimal(str(last_valid_price))
                    latest_prices[asset] = price # Store the price used
                except (IndexError, InvalidOperation, ValueError) as e:
                    print(f"  ⚠️ Could not get valid last close price for {symbol} from {PRICE_SOURCE_INTERVAL} klines ({e}). Will try ticker fallback.")
                    price = None # Ensure price is None if error occurs
            else:
                 print(f"  Info: Kline data for {symbol} ({PRICE_SOURCE_INTERVAL}) empty or missing 'Close'. Will try ticker fallback.")
                 price = None
        else:
             print(f"  Info: Kline data for {symbol} ({PRICE_SOURCE_INTERVAL}) unavailable. Will try ticker fallback.")
             price = None # Will trigger fallback below

        # Fallback: Try fetching ticker price if kline data failed or unavailable
        if price is None:
            print(f"  Attempting ticker fetch for {symbol}...")
            if 'client' in locals() and client is not None:
                try:
                    ticker = client.get_symbol_ticker(symbol=symbol)
                    price = Decimal(ticker['price'])
                    latest_prices[asset] = price
                    print(f"    -> Fetched ticker price: {price}")
                except Exception as e:
                     print(f"  ⚠️ Error fetching ticker for {symbol}: {e}. Skipping value.")
                     price = None
            else:
                 print(f"  ⚠️ Client not available to fetch ticker for {symbol}. Skipping value.")
                 price = None

        # Calculate value if price is valid
        if price is not None and price > Decimal('0.0'):
            value = total_balance * price
            current_portfolio_value += value
            current_allocations_value[asset] = value
            print(f"  {asset} Balance: {total_balance:.{DISPLAY_PRECISION_QTY}f} @ {price:.4f} = {value:.{DISPLAY_PRECISION_VAL}f} {QUOTE_ASSET}")
        else:
            # If we have balance but couldn't get price, add asset with 0 value
             current_allocations_value[asset] = Decimal('0.0')
             latest_prices[asset] = None # Ensure price is marked as unknown
             print(f"  {asset} Balance: {total_balance:.{DISPLAY_PRECISION_QTY}f} (Price Unknown - Value treated as 0)")

    print(f"\nTotal Portfolio Value: {current_portfolio_value:.{DISPLAY_PRECISION_VAL}f} {QUOTE_ASSET}")

    # Handle case of zero portfolio value
    if current_portfolio_value <= Decimal('0.0'):
        print("⚠️ Total portfolio value is zero or negative. Cannot calculate deviations.")
    else:
        # --- 2. Calculate Deviations and Ideal Trades ---
        print("\nDetermining ideal trades based on deviation...")
        trades_to_consider = []

        # Combine all assets: those in balance AND those in target list
        all_assets = set(current_allocations_value.keys()) | set(target_allocations_pct.keys())

        for asset in all_assets:
            current_value = current_allocations_value.get(asset, Decimal('0.0'))
            target_pct = target_allocations_pct.get(asset, Decimal('0.0')) # Get target %, default 0

            # Ensure target_pct is a valid Decimal
            if not isinstance(target_pct, Decimal):
                try:
                    target_pct = Decimal(str(target_pct))
                    print(f"  Warning: Converted target_pct for {asset} to Decimal.")
                except (InvalidOperation, ValueError):
                    print(f"  ERROR: Invalid target percentage type for {asset} ('{target_allocations_pct.get(asset)}'). Skipping.")
                    continue

            target_value = current_portfolio_value * (target_pct / Decimal('100.0'))
            deviation_value = current_value - target_value
            current_pct = (current_value / current_portfolio_value) * Decimal('100.0') if current_portfolio_value > Decimal('0.0') else Decimal('0.0')
            deviation_pct = current_pct - target_pct

            # Determine action (BUY if underweight, SELL if overweight)
            action = None
            # Use a small tolerance based on MIN_TRADE_VALUE to avoid tiny trades near zero deviation
            tolerance = MIN_TRADE_VALUE_USDT / Decimal('2.0')
            if deviation_value < -tolerance: # Significantly underweight -> BUY
                action = 'BUY'
            elif deviation_value > tolerance: # Significantly overweight -> SELL
                action = 'SELL'

            # Calculate ideal quantity only if action is needed
            ideal_quantity = Decimal('0.0')
            price_used = latest_prices.get(asset) # Get price stored earlier

            # Ensure we have a valid price for quantity calculation
            if action and (price_used is None or price_used <= Decimal('0.0')):
                 # If price was unknown earlier, try fetching ticker price again now
                 symbol = f"{asset}{QUOTE_ASSET}" if asset != QUOTE_ASSET else None
                 print(f"  Attempting price re-fetch for {asset} (needed for quantity)...")
                 if symbol and 'client' in locals() and client is not None:
                      try:
                          ticker = client.get_symbol_ticker(symbol=symbol)
                          price_used = Decimal(ticker['price'])
                          latest_prices[asset] = price_used # Store the newly fetched price
                          print(f"    -> Fetched ticker price: {price_used}")
                      except Exception as e:
                           print(f"    ⚠️ Error fetching ticker for {symbol}: {e}. Cannot calculate quantity.")
                           price_used = None
                 elif asset == QUOTE_ASSET:
                      price_used = Decimal('1.0') # Price for USDT is 1
                 else:
                      price_used = None # Could not fetch

            # Calculate quantity if price is valid
            if action and price_used is not None and price_used > Decimal('0.0'):
                # Ideal quantity is the absolute value of the deviation, divided by price
                ideal_quantity = abs(deviation_value) / price_used
            elif action:
                # Cannot determine quantity if price is invalid
                print(f"  ⚠️ Cannot calculate ideal quantity for {action} {asset} due to invalid/missing price.")
                action = None # Nullify action if quantity cannot be calculated

            # Add to list if trade action is defined and meets minimum value threshold
            # Use absolute deviation value for the threshold check
            if action and abs(deviation_value) >= MIN_TRADE_VALUE_USDT:
                trades_to_consider.append({
                    'Asset': asset,
                    'Action': action,
                    'IdealQuantity': ideal_quantity, # Store as Decimal
                    'IdealValueUSDT': abs(deviation_value), # Absolute value of deviation
                    'CurrentValueUSDT': current_value,
                    'TargetValueUSDT': target_value,
                    'DeviationValueUSDT': deviation_value,
                    'DeviationPercent': deviation_pct,
                    'CurrentPct': current_pct,
                    'TargetPct': target_pct,
                    'CalcPrice': price_used # Store the price used for calculation
                })
            elif action:
                 # Trade doesn't meet min value or tolerance, log for info
                 print(f"  Skipping {action} {asset}: Deviation value {abs(deviation_value):.2f} < Min Trade Value {MIN_TRADE_VALUE_USDT:.2f} or within tolerance.")


        # --- 3. Create and Sort DataFrame ---
        if trades_to_consider:
            trades_df = pd.DataFrame(trades_to_consider)
            # Sort: SELLs first, then BUYs (prioritize larger deviations within each group)
            trades_df.sort_values(by=['Action', 'IdealValueUSDT'], ascending=[False, False], inplace=True)
            trades_df.reset_index(drop=True, inplace=True)
            calculation_success = True
        else:
            print("No significant deviations found requiring trades.")
            calculation_success = True # Still successful, just no trades needed

except Exception as e:
    print(f"❌ An error occurred during ideal trade calculation: {e}")
    import traceback
    traceback.print_exc() # Print detailed traceback for debugging
    # Ensure trades_df is empty on error
    trades_df = pd.DataFrame(columns=trades_df.columns)
    calculation_success = False

# --- Final Output ---
if calculation_success and not trades_df.empty:
    print("\n--- Ideal Rebalancing Trades Proposed ---")
    # Format for display
    display_df = trades_df[['Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT', 'CalcPrice']].copy()
    # Define formatters using Decimal/float check and specified precision
    def fmt_qty(x): return f"{x:.{DISPLAY_PRECISION_QTY}f}" if isinstance(x, (Decimal, float)) else str(x)
    def fmt_val(x): return f"{x:.{DISPLAY_PRECISION_VAL}f}" if isinstance(x, (Decimal, float)) else str(x)
    # Show price with more precision in output table if desired
    def fmt_price(x): return f"{x:.6f}" if isinstance(x, (Decimal, float)) else str(x)

    formatters = {
        'IdealQuantity': fmt_qty,
        'IdealValueUSDT': fmt_val,
        'CalcPrice': fmt_price
    }
    print(display_df.to_string(index=False, formatters=formatters))
    print(f"\n✅ Ideal trades calculation finished (Success={calculation_success}). Found {len(trades_df)} trades.")
    print("   Stored results in 'trades_df'.")
elif calculation_success:
    print("\n✅ Ideal trades calculation finished (Success=True). No trades met the minimum threshold.")
    print("   'trades_df' is empty.")
else:
    print("\n❌ Ideal trades calculation FAILED.")
    print("   'trades_df' is empty.")

# Ensure df exists even if calculation failed or yielded no trades
if 'trades_df' not in locals():
    trades_df = pd.DataFrame(columns=[
    'Asset', 'Action', 'IdealQuantity', 'IdealValueUSDT', 'CurrentValueUSDT',
    'TargetValueUSDT', 'DeviationValueUSDT', 'DeviationPercent', 'CurrentPct',
    'TargetPct', 'CalcPrice'])


--- Determining Ideal Rebalancing Trades (Based on Total Value Deviation) ---
Calculating current portfolio value...
  USDT Balance: 8.97
  BTC Balance: 0.00005710 @ 78835.3000 = 4.50 USDT
  ETH Balance: 0.00318720 @ 1485.0000 = 4.73 USDT
  XRP Balance: 5.97600000 @ 1.8595 = 11.11 USDT
  ADA Balance: 1.99200000 @ 0.5725 = 1.14 USDT
  Info: Kline data for BUSDUSDT (5m) unavailable. Will try ticker fallback.
  Attempting ticker fetch for BUSDUSDT...
    -> Fetched ticker price: 0.99930000
  BUSD Balance: 0.14956400 @ 0.9993 = 0.15 USDT
  Info: Kline data for WAVESUSDT (5m) unavailable. Will try ticker fallback.
  Attempting ticker fetch for WAVESUSDT...
    -> Fetched ticker price: 3.32900000
  WAVES Balance: 0.01000000 @ 3.3290 = 0.03 USDT
  SOL Balance: 0.02988000 @ 103.7400 = 3.10 USDT

Total Portfolio Value: 33.74 USDT

Determining ideal trades based on deviation...
  Skipping SELL XRP: Deviation value 8.41 < Min Trade Value 10.00 or within tolerance.
No significant deviations found

In [17]:
# Cell 17 (Corrected BUY Feasibility Logic): Check Trade Feasibility

import pandas as pd
import numpy as np
from decimal import Decimal

# --- Configuration ---\
QUOTE_ASSET = 'USDT'

# --- Check Prerequisites ---\
if 'trades_df' not in locals() or trades_df.empty:
     print("\n'trades_df' is empty/missing. Skipping feasibility check.")
     feasible_trades_df = pd.DataFrame()
elif 'portfolio_state' not in locals() or 'balances' not in portfolio_state or 'open_orders' not in portfolio_state:
     raise ValueError("Portfolio state missing. Run Cell 2 first.")
else:
    print("\n--- Checking Trade Feasibility Against Free Balances & Open Orders ---")

    feasible_trades_df = trades_df.copy()
    feasible_trades_df['Feasible'] = True
    feasible_trades_df['ConflictType'] = ''
    feasible_trades_df['BlockingOrderIDs'] = [[] for _ in range(len(feasible_trades_df))]

    balances = portfolio_state['balances']
    open_orders = portfolio_state['open_orders']

    free_usdt = balances.loc[QUOTE_ASSET, 'Free'] if QUOTE_ASSET in balances.index else Decimal('0.0')
    print(f"Available Free {QUOTE_ASSET} (from Cell 2): {free_usdt:.8f}")

    # --- Track cumulative USDT needed for BUYS ---
    cumulative_buy_cost = Decimal('0.0')

    # Iterate through proposed trades (ensure consistent order, e.g., SELLs first)
    # Assuming trades_df is already sorted SELLs then BUYs from Cell 16
    for index, trade in feasible_trades_df.iterrows():
        asset = trade['Asset']; action = trade['Action']
        ideal_qty = Decimal(str(trade['IdealQuantity']))
        ideal_value = Decimal(str(trade['IdealValueUSDT'])) # Value of this specific trade

        # --- Check SELL Feasibility vs FREE balance ---
        if action == 'SELL':
            free_balance = balances.loc[asset,'Free'] if asset in balances.index else Decimal('0.0')
            if ideal_qty > free_balance:
                feasible_trades_df.loc[index, 'Feasible'] = False
                feasible_trades_df.loc[index, 'ConflictType'] = 'Insufficient Free Balance'
                symbol_str = f"{asset}{QUOTE_ASSET}" if asset != QUOTE_ASSET else None
                blocking_orders = []
                if symbol_str and not open_orders.empty and 'symbol' in open_orders:
                    blocking_orders = open_orders[ (open_orders['symbol']==symbol_str) & (open_orders['side']=='SELL') & (open_orders['status'].isin(['NEW','PARTIALLY_FILLED'])) ]['orderId'].tolist()
                feasible_trades_df.at[index, 'BlockingOrderIDs'] = blocking_orders

        # --- Check BUY Feasibility vs FREE USDT (Cumulative) ---
        elif action == 'BUY':
            # --- CORRECTED LOGIC ---
            required_usdt_for_this_buy = ideal_value
            # Check if *this* order PLUS previous BUYs exceed available free USDT
            if cumulative_buy_cost + required_usdt_for_this_buy > free_usdt:
                feasible_trades_df.loc[index, 'Feasible'] = False
                feasible_trades_df.loc[index, 'ConflictType'] = f'Insufficient Free {QUOTE_ASSET} (Cumulative)'
                # List open BUY orders potentially locking USDT
                blocking_orders = []
                if not open_orders.empty and 'side' in open_orders:
                     blocking_orders = open_orders[ (open_orders['side'] == 'BUY') & (open_orders['status'].isin(['NEW', 'PARTIALLY_FILLED'])) ]['orderId'].tolist()
                feasible_trades_df.at[index, 'BlockingOrderIDs'] = blocking_orders
            else:
                # If this BUY is feasible, add its cost to the cumulative total
                cumulative_buy_cost += required_usdt_for_this_buy
            # --- END CORRECTED LOGIC ---\

    # --- Report Summary ---\
    feasible_count = feasible_trades_df['Feasible'].sum()
    conflicting_count = len(feasible_trades_df) - feasible_count
    print(f"\nFeasibility Check Summary:"); print(f"  Feasible: {feasible_count}, Conflicting: {conflicting_count}")
    if conflicting_count > 0:
        print("\n--- Conflicting Trades Details ---")
        cols = ['Action','Asset','IdealQuantity','IdealValueUSDT','ConflictType','BlockingOrderIDs']
        cols = [c for c in cols if c in feasible_trades_df.columns]
        def fmt(x,p=8): return f"{x:.{p}f}" if isinstance(x,Decimal) else (f"{x:.{p}f}" if isinstance(x,(int,float)) else str(x))
        formatters = {'IdealQuantity':lambda x:fmt(x,8), 'IdealValueUSDT':lambda x:fmt(x,2)}
        print(feasible_trades_df.loc[~feasible_trades_df['Feasible'], cols].to_string(formatters=formatters, index=False))

    print(f"\n✅ Feasibility check complete. Stored in 'feasible_trades_df'.")

if 'feasible_trades_df' not in locals(): feasible_trades_df = pd.DataFrame()


'trades_df' is empty/missing. Skipping feasibility check.


In [18]:
# Cell 18: Cancel Conflicting Orders
# This cell cancels existing open orders that conflict with proposed trades.

import pandas as pd
from binance.client import Client
from decimal import Decimal

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client not initialized. Run Cell 2 first.")
if 'feasible_trades_df' not in locals():
    print("⚠️ 'feasible_trades_df' not found. No orders to cancel.")
    cancellation_summary = {"attempted": 0, "successful": 0, "errors": 0}
elif feasible_trades_df.empty:
    print("ℹ️ 'feasible_trades_df' is empty. No orders to cancel.")
    cancellation_summary = {"attempted": 0, "successful": 0, "errors": 0}
elif 'portfolio_state' not in locals() or 'open_orders' not in portfolio_state:
    raise ValueError("Portfolio state missing 'open_orders'. Run Cell 2 first.")
else:
    print("\n--- Cancelling Conflicting Open Orders ---")
    open_orders = portfolio_state['open_orders']
    cancellation_summary = {"attempted": 0, "successful": 0, "errors": 0}

    # 1. Extract Symbols from Trades
    symbols_to_check = set()
    for index, trade in feasible_trades_df.iterrows():
        asset = trade['Asset']
        symbol = f"{asset}{QUOTE_ASSET}" if asset != QUOTE_ASSET else QUOTE_ASSET # Include quote asset symbol for cancels
        symbols_to_check.add(symbol)

    # Convert to a list for easier iteration and to avoid modifying the set during iteration.
    symbols_to_check_list = list(symbols_to_check)

    # 2. Iterate through Open Orders and Cancel Conflicting Ones
    if not open_orders.empty and 'symbol' in open_orders.columns and not feasible_trades_df.empty:
        print(f"  Found {len(open_orders)} open order(s).")
        for index, order in open_orders.iterrows():
            symbol = order['symbol']
            order_id = order['orderId']

            # Cancel only orders for symbols relevant to our trade strategy
            if symbol in symbols_to_check_list:
                cancellation_summary["attempted"] += 1
                try:
                    print(f"    Attempting to cancel order {order_id} for {symbol}...")
                    result = client.cancel_order(symbol=symbol, orderId=order_id)
                    print(f"      ✅ Order {order_id} cancellation successful: {result}")
                    cancellation_summary["successful"] += 1
                except Exception as e:
                    print(f"      ❌ Error cancelling order {order_id} for {symbol}: {e}")
                    cancellation_summary["errors"] += 1
    else:
        print("  No open orders found, or no symbols to check against.")

    print(f"\nCancellation Summary: Attempted={cancellation_summary['attempted']}, Successful={cancellation_summary['successful']}, Errors={cancellation_summary['errors']}")

# Store the cancellation summary even if no cancels were attempted.
if 'cancellation_summary' not in locals():
    cancellation_summary = {"attempted": 0, "successful": 0, "errors": 0}
print("\nCancellation details stored in 'cancellation_summary'.")

# Ensure summary exists even if no cancellations occurred
if 'cancellation_summary' not in locals():
    cancellation_summary = {"attempted": 0, "successful": 0, "errors": 0}

ℹ️ 'feasible_trades_df' is empty. No orders to cancel.

Cancellation details stored in 'cancellation_summary'.


In [19]:
# Cell 19: Filter for Feasible Trades ONLY

import pandas as pd
from decimal import Decimal

# --- Check Prerequisites ---
if 'feasible_trades_df' not in locals():
    print("⚠️ 'feasible_trades_df' not found. Skipping filtering.")
    final_trades_to_format_df = pd.DataFrame()
elif feasible_trades_df.empty:
    print("ℹ️ 'feasible_trades_df' is empty. No trades to filter.")
    final_trades_to_format_df = pd.DataFrame()
else:
    print("\n--- Filtering for Feasible Trades (Based on Cell 17 check) ---")

    # Ensure necessary columns exist
    required_cols = ['Feasible', 'Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT']
    if not all(col in feasible_trades_df.columns for col in required_cols):
         missing = [c for c in required_cols if c not in feasible_trades_df.columns]
         raise ValueError(f"Input 'feasible_trades_df' missing columns: {missing}")

    # Filter condition: ONLY Feasible == True
    feasible_condition = (feasible_trades_df['Feasible'] == True)

    final_trades_to_format_df = feasible_trades_df[feasible_condition].copy()

    # --- Report Summary ---
    original_count = len(feasible_trades_df)
    proceed_count = len(final_trades_to_format_df)
    conflicting_count = original_count - proceed_count

    print(f"Out of {original_count} ideal trades considered:")
    print(f"  - {proceed_count} trade(s) are feasible based on current free balances.")
    if conflicting_count > 0:
        print(f"  - {conflicting_count} trade(s) were filtered out as infeasible.")

    if not final_trades_to_format_df.empty:
        print("\n--- Trades Ready for Formatting ---")
        display_cols = ['Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT']
        display_cols = [c for c in display_cols if c in final_trades_to_format_df.columns]
        def fmt(x,p=8): return f"{x:.{p}f}" if isinstance(x,Decimal) else (f"{x:.{p}f}" if isinstance(x,(int,float)) else str(x))
        formatters = {'IdealQuantity':lambda x:fmt(x,8), 'IdealValueUSDT':lambda x:fmt(x,2)}
        print(final_trades_to_format_df[display_cols].to_string(formatters=formatters, index=False))
        print("\n✅ Filtering complete. Feasible trades stored in 'final_trades_to_format_df'.")
    else:
        print("\n✅ Filtering complete. No feasible trades found to format.")


# Ensure the final DataFrame exists, even if empty
if 'final_trades_to_format_df' not in locals():
     final_trades_to_format_df = pd.DataFrame()

ℹ️ 'feasible_trades_df' is empty. No trades to filter.


In [20]:
# Cell 20: Get Updated Balances Post-Cancellation

import pandas as pd
from decimal import Decimal, ROUND_DOWN
import time

# --- Configuration ---
QUOTE_ASSET = 'USDT'
DISPLAY_PRECISION = 8
# Delay even if no cancellations attempted, ensures consistency in timing
POST_CANCEL_DELAY_SECONDS = 1.0 # Shorter delay might be fine if no cancels usually happen

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client not initialized. Run Cell 2 first.")
if 'get_current_balances_detailed' not in locals():
     raise RuntimeError("Function 'get_current_balances_detailed' not defined.")

# --- Add Delay ---
print(f"\nWaiting {POST_CANCEL_DELAY_SECONDS} seconds before final balance check...")
time.sleep(POST_CANCEL_DELAY_SECONDS)

print("\n--- Fetching Updated Balances Post-Potential-Cancellation ---")

updated_balances_df = pd.DataFrame() # Initialize empty
try:
    # Call the detailed balance function again
    updated_balances_df = get_current_balances_detailed(client) # Assumes function defined in Cell 1/2

    if not updated_balances_df.empty:
        print("✅ Updated balances fetched successfully.")
        print("\n--- Latest Balances (Free/Locked) ---")
        # Display with formatting
        balances_display_df = updated_balances_df.copy()
        formatter = f'{{:.{DISPLAY_PRECISION}f}}'
        for col in ['Free', 'Locked']:
             if col in balances_display_df.columns:
                  balances_display_df[col] = balances_display_df[col].apply(lambda x: formatter.format(x) if isinstance(x, Decimal) else x)
        print(balances_display_df)
    else:
        print("⚠️ Updated balances fetch returned empty or failed.")

except Exception as e:
    print(f"❌ Error fetching updated balances: {e}")
    updated_balances_df = pd.DataFrame() # Ensure empty on error

# Result stored in 'updated_balances_df'
if 'updated_balances_df' not in locals() or updated_balances_df.empty:
     print("\n⚠️ Could not get updated balances. Final order formatting may fail.")
else:
     print("\n✅ Updated balances stored in 'updated_balances_df'. Ready for final formatting.")


Waiting 1.0 seconds before final balance check...

--- Fetching Updated Balances Post-Potential-Cancellation ---
✅ Updated balances fetched successfully.

--- Latest Balances (Free/Locked) ---
             Free      Locked
Asset                        
BTC    0.00005710  0.00000000
ETH    0.00318720  0.00000000
XRP    5.97600000  0.00000000
USDT   8.96644535  0.00000000
ADA    1.99200000  0.00000000
BUSD   0.14956400  0.00000000
WAVES  0.01000000  0.00000000
SOL    0.02988000  0.00000000

✅ Updated balances stored in 'updated_balances_df'. Ready for final formatting.


In [21]:
# Cell 21: Format & Finalize Executable Buy Orders

import pandas as pd
import os
import numpy as np
# Need Client for exchange info, client should be initialized
from binance.client import Client
from decimal import Decimal, ROUND_DOWN

# --- Configuration ---
QUOTE_ASSET = 'USDT'
# Maker strategy (used if price needs re-formatting, though less likely for deep limits)
PLACE_BUY_AT = 'BID'
PLACE_SELL_AT = 'ASK' # Not used this cycle

# --- Check Prerequisites ---
if 'client' not in locals() or client is None: raise RuntimeError("Binance client not initialized.")
if 'format_price_correctly' not in locals() or 'adjust_quantity_to_step' not in locals(): raise RuntimeError("Formatting helpers not defined.")
# Use the buy orders defined in Cell 17
if 'buy_orders_to_place_df' not in locals(): print("⚠️ Defined buy orders ('buy_orders_to_place_df') not found."); final_executable_orders = []
elif buy_orders_to_place_df.empty: print("ℹ️ Defined buy orders list ('buy_orders_to_place_df') is empty."); final_executable_orders = []
# Use the LATEST balances from Cell 19
elif 'updated_balances_df' not in locals(): print("⚠️ Updated balances ('updated_balances_df') not found."); final_executable_orders = []
# Allow proceeding if balances df is technically present but empty
# elif updated_balances_df.empty: print("⚠️ Updated balances DataFrame is empty."); final_executable_orders = []
else:
    print("\n--- Formatting Final Executable BUY Orders ---")
    print(f"Checking against updated balances. Strategy: Place BUY orders at LLM suggested levels.")

    # --- Get Exchange Info ---
    print("Fetching exchange information for formatting rules...")
    symbols_info = None
    try:
        exchange_info = client.get_exchange_info(); symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
        print("✅ Exchange information received.")
    except Exception as e: print(f"❌ Error fetching exchange info: {e}. Formatting may be inaccurate.")

    final_executable_orders = [] # List for final parameters

    # Get updated free USDT balance from Cell 19 DataFrame
    free_usdt = updated_balances_df.loc[QUOTE_ASSET, 'Free'] if QUOTE_ASSET in updated_balances_df.index else Decimal('0.0')
    print(f"Updated Free {QUOTE_ASSET} for final check: {free_usdt:.8f}")

    usdt_to_be_spent = Decimal('0.0') # Track cumulative cost

    # Iterate through the BUY orders defined in Cell 17
    for index, order_def in buy_orders_to_place_df.iterrows():
        symbol = order_def['symbol']
        asset = symbol.replace(QUOTE_ASSET, '')
        side = order_def['side'] # Should be 'BUY'
        # Price/Qty from the definition step (Cell 17)
        target_price_dec = order_def['target_price'] # Decimal
        ideal_quantity_dec = order_def['ideal_quantity'] # Decimal
        intended_usdt_value = order_def['usdt_value'] # Decimal ($ per order config)

        print(f"\nProcessing Defined Order {index+1}: {side} {asset} @ {target_price_dec:.8f} (Qty: ~{ideal_quantity_dec:.8f})")

        # --- Final Feasibility Check vs Updated Balances ---
        # Check if we have enough USDT for THIS order, considering already allocated USDT
        if usdt_to_be_spent + intended_usdt_value > free_usdt:
            print(f"  INFEASIBLE: Need ~${intended_usdt_value:.2f} {QUOTE_ASSET}, Available after prior allocations: ${(free_usdt - usdt_to_be_spent):.2f}. Skipping.")
            continue # Skip this order and potentially subsequent ones if using priority
        else:
             print(f"  FEASIBLE: Sufficient USDT available for this order.")
             # Tentatively commit the USDT for this order for subsequent checks
             usdt_to_be_spent += intended_usdt_value

        # --- Formatting ---
        if symbols_info is None or symbol not in symbols_info: print(f"  ⚠️ Rules not found for {symbol}. Skipping formatting."); continue
        symbol_info = symbols_info[symbol]; filters = {f['filterType']: f for f in symbol_info['filters']}
        price_filter=filters.get('PRICE_FILTER',{}); lot_size_filter=filters.get('LOT_SIZE',{})
        notional_filter=filters.get('NOTIONAL',{}); min_notional_str = notional_filter.get('minNotional')
        if min_notional_str is None: min_notional_str = filters.get('MIN_NOTIONAL',{}).get('minNotional','0.0')
        tick_size=price_filter.get('tickSize'); step_size=lot_size_filter.get('stepSize')

        if not tick_size or not step_size or min_notional_str is None: print(f"  ⚠️ Missing filter info. Skipping."); continue
        try: min_notional = Decimal(min_notional_str)
        except Exception: print(f"  ⚠️ Invalid minNotional. Skipping."); continue

        try:
            # Format the TARGET price provided by LLM/Cell 17
            adjusted_price_str = format_price_correctly(target_price_dec, tick_size)
            # Adjust the IDEAL quantity calculated in Cell 17 based on step size
            adjusted_quantity_str = adjust_quantity_to_step(ideal_quantity_dec, step_size)
            # Convert back for final check
            adjusted_price_dec_final = Decimal(adjusted_price_str)
            adjusted_quantity_dec_final = Decimal(adjusted_quantity_str)
            print(f"  Formatted Price Str: {adjusted_price_str}")
            print(f"  Adjusted Quantity Str: {adjusted_quantity_str}")
        except Exception as fmt_err: print(f"  ❌ Formatting error: {fmt_err}. Skipping."); continue

        # Final Check: Adjusted Quantity > 0 and Meets MIN_NOTIONAL
        if adjusted_quantity_dec_final <= 0: print(f"  ⚠️ Adjusted quantity zero or less. Skipping."); continue
        order_value_final = adjusted_quantity_dec_final * adjusted_price_dec_final
        if min_notional > 0 and order_value_final < min_notional:
             print(f"  ⚠️ Final order value ({order_value_final:.8f}) < min notional ({min_notional:.8f}). Skipping.");
             # Since skipped, roll back the USDT commitment for this order
             usdt_to_be_spent -= intended_usdt_value
             continue

        print(f"  ✅ Final order meets min notional ({order_value_final:.8f} >= {min_notional:.8f}).")

        # Add Validated Order Parameters
        final_executable_orders.append({
            'symbol': symbol, 'side': side.upper(), 'type': 'LIMIT', 'timeInForce': 'GTC',
            'quantity': adjusted_quantity_str, # FORMATTED STRING
            'price': adjusted_price_str   # FORMATTED STRING
        })
        print(f"  ✅ Added valid LIMIT order parameters for {symbol} to final execution list.")

# --- Final Display ---
print("\n--- Final Executable LIMIT Orders ---")
if 'final_executable_orders' not in locals() or not final_executable_orders:
    print("No buy orders were feasible or met formatting requirements.")
    final_executable_orders = [] # Ensure list exists and is empty
else:
    executable_orders_df = pd.DataFrame(final_executable_orders)
    print(executable_orders_df[['symbol', 'side', 'type', 'quantity', 'price']])
    print(f"\nStored {len(final_executable_orders)} order(s) in 'final_executable_orders'.")

# Ensure list exists
if 'final_executable_orders' not in locals(): final_executable_orders = []

⚠️ Defined buy orders ('buy_orders_to_place_df') not found.

--- Final Executable LIMIT Orders ---
No buy orders were feasible or met formatting requirements.


In [22]:
# Cell 22 (Corrected String Literal): Final LLM Check & User Confirmation

import pandas as pd
import json
import time
from decimal import Decimal
# Ensure google.generativeai is imported if needed and configured
try: import google.generativeai as genai
except ImportError: print("Warning: google.generativeai not found."); genai = None
from dotenv import load_dotenv

# --- Configuration ---
EXECUTE_REAL_ORDERS_FLAG = True # Controls whether Cell 22 runs, but confirmation happens here
REQUIRE_CONFIRMATION = True
PERFORM_LLM_FINAL_CHECK = True
# --- CORRECTED LINE ---
LLM_MODEL_NAME_FINAL_CHECK = "gemini-1.5-flash-latest" # Or "gemini-pro"
# --- END CORRECTION ---
QUOTE_ASSET = 'USDT'
RSI_PERIOD = 14

# --- Initialize flag ---
proceed_user_confirmation = False # Default to not proceeding

# --- Check Prerequisites ---
# Check the final list from Cell 20
if 'final_executable_orders' not in locals():
     print("⚠️ 'final_executable_orders' list not found. Cannot proceed.")
elif not final_executable_orders:
     print("ℹ️ 'final_executable_orders' is empty. Nothing to execute.")
# Check LLM requirements only if check is enabled
elif PERFORM_LLM_FINAL_CHECK and (genai is None or 'genai_configured' not in locals() or not genai_configured):
     print("⚠️ LLM not configured/imported. Disabling final LLM check.")
     PERFORM_LLM_FINAL_CHECK = False
elif PERFORM_LLM_FINAL_CHECK and 'full_market_state' not in locals():
     print("⚠️ Market state not found. Disabling final LLM check."); PERFORM_LLM_FINAL_CHECK = False
elif PERFORM_LLM_FINAL_CHECK and 'news_summary' not in locals():
     print("⚠️ News summary not found. Disabling final LLM check."); PERFORM_LLM_FINAL_CHECK = False
# If all checks pass or LLM check disabled:
else:
    # --- Display Orders First ---
    print("\n" + "="*60); print(" !!! FINAL CONFIRMATION BEFORE LIVE EXECUTION !!!"); print("="*60)
    print("The following order(s) will be placed if confirmed:")
    try:
        orders_to_exec_df = pd.DataFrame(final_executable_orders); print(orders_to_exec_df[['symbol', 'side', 'type', 'quantity', 'price']])
        print(f"\nTotal order(s) to execute: {len(orders_to_exec_df)}")
    except Exception as e: print(f"Error previewing orders: {e}\nRaw list:", final_executable_orders)
    print("="*60)

    # --- Perform Final LLM Sanity Check (if enabled) ---
    llm_final_recommendation = "APPROVE"; llm_final_rationale = "LLM check skipped or default."
    if PERFORM_LLM_FINAL_CHECK:
        print("\n--- Performing Final LLM Sanity Check ---")
        try:
            # Prepare Context
            orders_str_list=[]; symbols_involved = set()
            for order in final_executable_orders: orders_str_list.append(f"- {order['side']} {order['quantity']} {order['symbol']} @ {order['price']}"); symbols_involved.add(order['symbol'])
            orders_for_prompt = "\n".join(orders_str_list); ta_summary_parts = []
            macro_context = locals().get('MACRO_CONTEXT_SUMMARY', "Not specified.") # Get if defined, else default
            for sym in symbols_involved:
                state = full_market_state.get(sym)
                if state and not state.get('error'):
                    trend = 'UP' if state.get('daily_trend_up') else ('DOWN' if state.get('daily_trend_up') is False else 'Unk'); rsi_val = state.get('last_5m_rsi'); rsi_str = f"{rsi_val:.1f}" if rsi_val is not None else "N/A"
                    ta_summary_parts.append(f"{sym}: Trend {trend}, RSI {rsi_str}")
                else: ta_summary_parts.append(f"{sym}: TA N/A")
            ta_summary_for_prompt = "; ".join(ta_summary_parts)
            news_context_summary = news_summary.get('summary', 'N/A'); news_context_sentiment = news_summary.get('sentiment', 'Unknown')

            # Construct Final Check Prompt
            final_check_prompt = f"""
            Perform a final review for a batch of cryptocurrency trades generated by a long-term accumulation/rebalancing strategy.

            CONTEXT:
            * Strategy Goal: Long-term growth, buy dips in core assets, rebalance toward volume-based targets.
            * Recent News Sentiment: {news_context_sentiment}
            * Recent News Summary: {news_context_summary}
            * Relevant Asset TA Summary: {ta_summary_for_prompt}
            * Macro Environment Context: {macro_context}

            PROPOSED ORDERS (Calculated, Feasible, Formatted):
            {orders_for_prompt}

            FINAL REVIEW TASK:
            Considering the strategy, context, orders, and your general market knowledge (history, tokenomics):
            1. Briefly summarize your assessment of executing this batch *now* (1-2 sentences). Is it generally aligned with the strategy and current context, or are there significant contradictions/red flags?
            2. State your final recommendation clearly on a new line: **APPROVE EXECUTION** or **REJECT EXECUTION**.

            Example Output:
            Assessment: The proposed BUY order for BTC aligns with the accumulation strategy, targeting a potential support level identified. News context is mixed, TA is neutral. No major red flags.
            Recommendation: APPROVE EXECUTION
            """

            # Call LLM
            model = genai.GenerativeModel(LLM_MODEL_NAME_FINAL_CHECK)
            safety_settings=[{"category": c, "threshold": "BLOCK_NONE"} for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]]
            response = model.generate_content(final_check_prompt, safety_settings=safety_settings)

            if not response.candidates or (response.prompt_feedback and response.prompt_feedback.block_reason):
                 reason = response.prompt_feedback.block_reason if response.prompt_feedback else "Unknown"; print(f"LLM Final Check BLOCKED: {reason}")
                 llm_final_recommendation = "REJECT"; llm_final_rationale = f"LLM Response Blocked: {reason}"
            else:
                 raw_text = response.text.strip(); print(f"LLM Final Check Response:\n---\n{raw_text}\n---")
                 llm_final_rationale = raw_text
                 last_line_upper = raw_text.split('\n')[-1].upper()
                 if "REJECT EXECUTION" in last_line_upper: llm_final_recommendation = "REJECT"; print("LLM signaled REJECT.")
                 elif "APPROVE EXECUTION" in last_line_upper: llm_final_recommendation = "APPROVE"; print("LLM signaled APPROVE.")
                 else: llm_final_recommendation = "REJECT"; llm_final_rationale = "Recommendation keyword unclear"; print("⚠️ LLM recommendation keyword unclear, defaulting to REJECT.")

        except Exception as llm_err:
            print(f"❌ Error during LLM final check: {llm_err}"); llm_final_recommendation = "REJECT"; llm_final_rationale = f"LLM API Error: {llm_err}"

        print("-" * 30); print(f"LLM FINAL RECOMMENDATION: {llm_final_recommendation}"); print("-" * 30)
    else: print("\n--- Skipping Final LLM Sanity Check (Setting: OFF or Prereqs Missing) ---")

    # --- Get User Confirmation ---
    if REQUIRE_CONFIRMATION:
        print("\nDecision Point: Review the prepared order(s) and LLM recommendation above.")
        try:
            prompt_msg = f">>> Type 'yes' exactly to EXECUTE the prepared order(s) (LLM: {llm_final_recommendation}): "
            confirm = input(prompt_msg)
            if confirm == 'yes': proceed_user_confirmation = True; print("\nUser confirmed execution...")
            else: print("\nUser confirmation not given. EXECUTION CANCELLED.")
        except EOFError: print("\nCould not get user confirmation (EOFError). EXECUTION CANCELLED.")
    else: # Confirmation not required
        print("Skipping user confirmation step (REQUIRE_CONFIRMATION = False).")
        if PERFORM_LLM_FINAL_CHECK and llm_final_recommendation == "REJECT":
             print("EXECUTION HALTED: LLM recommended REJECT (and user confirmation was skipped)."); proceed_user_confirmation = False
        else: proceed_user_confirmation = True

# Store decision state for next cell
print(f"\n✅ Final check & confirmation complete. Ready to execute? {proceed_user_confirmation}")

# Ensure flag exists even if initial checks failed
if 'proceed_user_confirmation' not in locals(): proceed_user_confirmation = False
# Ensure LLM recommendation state exists
if 'llm_final_recommendation' not in locals(): llm_final_recommendation = "SKIP" # Or APPROVE if default desired when skipped

ℹ️ 'final_executable_orders' is empty. Nothing to execute.

✅ Final check & confirmation complete. Ready to execute? False


In [23]:
# Cell 23: Final LLM Check & User Confirmation

import pandas as pd
import json
import time
from decimal import Decimal
try: import google.generativeai as genai
except ImportError: print("Warning: google.generativeai not found."); genai = None
from dotenv import load_dotenv

# --- Configuration ---
EXECUTE_REAL_ORDERS_FLAG = True # User decision stored here now
REQUIRE_CONFIRMATION = True
PERFORM_LLM_FINAL_CHECK = True
LLM_MODEL_NAME_FINAL_CHECK = "gemini-1.5-flash-latest"
QUOTE_ASSET = 'USDT'
RSI_PERIOD = 14
DEFAULT_LLM_REC = "APPROVE" # Default if check skipped/fails

# --- Initialize flags ---
proceed_user_confirmation = False # Final user decision
llm_final_recommendation = DEFAULT_LLM_REC
llm_final_rationale = "LLM check skipped or default."

# --- Check Prerequisites --- # <<< MODIFIED BLOCK
execution_possible = False # Can we even execute anything?
llm_check_possible = PERFORM_LLM_FINAL_CHECK # Should we try the LLM check?

if 'client' not in locals() or client is None: print("❌ Client not initialized.")
elif 'format_price_correctly' not in locals(): print("❌ Formatting helpers missing.")
elif 'final_executable_orders' not in locals(): print("⚠️ 'final_executable_orders' list not found.")
elif not final_executable_orders: print("ℹ️ 'final_executable_orders' is empty.")
else: execution_possible = True # Basic execution prerequisites met

# Check LLM prereqs only if enabled AND execution is possible
if not execution_possible:
    llm_check_possible = False # No point checking if we can't execute
elif PERFORM_LLM_FINAL_CHECK and (genai is None or 'genai_configured' not in locals() or not genai_configured):
     print("⚠️ LLM not configured/imported. Disabling final LLM check.")
     llm_check_possible = False
elif PERFORM_LLM_FINAL_CHECK and 'full_market_state' not in locals():
     print("⚠️ Market state not found. Disabling final LLM check."); llm_check_possible = False
elif PERFORM_LLM_FINAL_CHECK and 'news_summary' not in locals():
     print("⚠️ News summary not found. Disabling final LLM check."); llm_check_possible = False
# --- END Check Prerequisites --- #

if not execution_possible:
     print("\nSkipping confirmation and execution steps as no orders are ready.")
else:
    # --- Display Orders First ---
    # (Display logic remains the same...)
    print("\n" + "="*60); print(" !!! FINAL CONFIRMATION BEFORE LIVE EXECUTION !!!"); print("="*60);
    try: orders_to_exec_df = pd.DataFrame(final_executable_orders); print(orders_to_exec_df[['symbol', 'side', 'type', 'quantity', 'price']]); print(f"\nTotal order(s) to execute: {len(orders_to_exec_df)}")
    except Exception as e: print(f"Error previewing: {e}\n{final_executable_orders}")
    print("="*60)

    # --- Perform Final LLM Sanity Check (if possible) ---
    if llm_check_possible:
        print("\n--- Performing Final LLM Sanity Check ---")
        try:
            # (Prepare context logic remains the same...)
            orders_str_list=[]; symbols_involved=set();[orders_str_list.append(f"- {o['side']} {o['quantity']} {o['symbol']} @ {o['price']}") or symbols_involved.add(o['symbol']) for o in final_executable_orders]
            orders_for_prompt = "\n".join(orders_str_list); ta_summary_parts = []
            macro_context = locals().get('MACRO_CONTEXT_SUMMARY', "Not specified.")
            for sym in symbols_involved:
                state = full_market_state.get(sym)
                if state and not state.get('error'): trend='UP' if state.get('daily_trend_up') else ('DOWN' if state.get('daily_trend_up') is False else 'Unk'); rsi_val=state.get('last_5m_rsi'); rsi_str=f"{rsi_val:.1f}" if rsi_val is not None else "N/A"; ta_summary_parts.append(f"{sym}: Trend {trend}, RSI {rsi_str}")
                else: ta_summary_parts.append(f"{sym}: TA N/A")
            ta_summary_for_prompt = "; ".join(ta_summary_parts)
            news_context_summary = news_summary.get('summary', 'N/A'); news_context_sentiment = news_summary.get('sentiment', 'Unknown')
            # (Construct prompt logic remains the same...)
            final_check_prompt = f"""Final Sanity Check...CONTEXT: Strategy Goal: Long-term growth... | Macro: {macro_context} | News Sentiment: {news_context_sentiment} | News Summary: {news_context_summary} | TA Summary: {ta_summary_for_prompt}\nPROPOSED ORDER(S):\n{orders_for_prompt}\nFINAL QUESTION: Any major red flags to abort this batch now?\nRespond: **APPROVE EXECUTION** or **REJECT EXECUTION**, optionally followed by brief reason."""
            # (Call LLM logic remains the same...)
            model = genai.GenerativeModel(LLM_MODEL_NAME_FINAL_CHECK)
            safety_settings=[{"category": c, "threshold": "BLOCK_NONE"} for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]]
            response = model.generate_content(final_check_prompt, safety_settings=safety_settings)
            if not response.candidates or (response.prompt_feedback and response.prompt_feedback.block_reason): reason = response.prompt_feedback.block_reason if response.prompt_feedback else "Unknown"; print(f"LLM Blocked: {reason}"); llm_final_recommendation = "REJECT"; llm_final_rationale = f"LLM Blocked: {reason}"
            else:
                 raw_text = response.text.strip(); print(f"LLM Final Check Response:\n---\n{raw_text}\n---"); llm_final_rationale = raw_text
                 last_line_upper = raw_text.split('\n')[-1].upper()
                 if "REJECT EXECUTION" in last_line_upper: llm_final_recommendation = "REJECT"; print("LLM signaled REJECT.")
                 elif "APPROVE EXECUTION" in last_line_upper: llm_final_recommendation = "APPROVE"; print("LLM signaled APPROVE.")
                 else: llm_final_recommendation = "REJECT"; llm_final_rationale = "Keyword unclear"; print("⚠️ LLM keyword unclear, defaulting to REJECT.")
        except Exception as llm_err: print(f"❌ Error during LLM final check: {llm_err}"); llm_final_recommendation = "REJECT"; llm_final_rationale = f"LLM API Error: {llm_err}"
        print("-" * 30); print(f"LLM FINAL RECOMMENDATION: {llm_final_recommendation}"); print("-" * 30)
    # Handle cases where check was disabled or skipped
    elif PERFORM_LLM_FINAL_CHECK: print("\n--- Skipping Final LLM Sanity Check (Prerequisites Missing) ---")
    else: print("\n--- Skipping Final LLM Sanity Check (Setting: OFF) ---")

    # --- Get User Confirmation ---
    if REQUIRE_CONFIRMATION:
        print("\nDecision Point: Review the prepared order(s) and LLM recommendation above.")
        try:
            prompt_msg = f">>> Type 'yes' exactly to EXECUTE the prepared order(s) (LLM: {llm_final_recommendation}): "
            confirm = input(prompt_msg)
            if confirm == 'yes': proceed_user_confirmation = True; print("\nUser confirmed execution...")
            else: print("\nUser confirmation not given. EXECUTION CANCELLED.")
        except EOFError: print("\nCould not get user confirmation (EOFError). EXECUTION CANCELLED.")
    else: # Confirmation not required
        print("Skipping user confirmation step (REQUIRE_CONFIRMATION = False).")
        if PERFORM_LLM_FINAL_CHECK and llm_final_recommendation == "REJECT": print("EXECUTION HALTED: LLM recommended REJECT."); proceed_user_confirmation = False
        else: proceed_user_confirmation = True # Auto-proceed if confirmation off and LLM didn't reject


# Final decision stored in 'proceed_user_confirmation' flag
print(f"\n✅ Final check & confirmation complete. Ready to execute? {proceed_user_confirmation}")
if 'proceed_user_confirmation' not in locals(): proceed_user_confirmation = False
if 'llm_final_recommendation' not in locals(): llm_final_recommendation = "SKIP"

ℹ️ 'final_executable_orders' is empty.

Skipping confirmation and execution steps as no orders are ready.

✅ Final check & confirmation complete. Ready to execute? False


In [24]:
# Cell 21: Final Feasibility Filter (Using Updated Balances)
# Re-evaluates the ideal trades from Cell 16 against the balances from Cell 20.

import pandas as pd
import numpy as np
from decimal import Decimal

# --- Configuration ---\
QUOTE_ASSET = 'USDT'
DISPLAY_PRECISION_QTY = 8
DISPLAY_PRECISION_VAL = 2

# --- Check Prerequisites ---\
# Use ideal trades from Cell 16
if 'trades_df' not in locals():
     print("⚠️ Ideal trades ('trades_df' from Cell 16) not found. Cannot filter.")
     final_trades_to_format_df = pd.DataFrame() # Cannot proceed
elif trades_df.empty:
     print("ℹ️ Ideal trades list ('trades_df') is empty. No trades to filter.")
     final_trades_to_format_df = pd.DataFrame() # Nothing to filter
# Use UPDATED balances from Cell 20
elif 'updated_balances_df' not in locals():
     print("⚠️ Updated balances ('updated_balances_df' from Cell 20) not found. Cannot check final feasibility.")
     # If updated balances failed, we cannot proceed safely.
     final_trades_to_format_df = pd.DataFrame()
elif updated_balances_df.empty:
      print("⚠️ Updated balances DataFrame ('updated_balances_df') is empty. Cannot check final feasibility.")
      # If updated balances are empty (e.g., API error), we cannot proceed safely.
      final_trades_to_format_df = pd.DataFrame()
else:
    print("\n--- Filtering Ideal Trades by Updated Feasibility (Cell 20 Balances) ---")

    # Copy ideal trades to start filtering
    filtered_df = trades_df.copy() # Work on a copy of the original ideal trades
    filtered_df['Feasible_Updated'] = True # Add new feasibility column
    filtered_df['Feasibility_Reason'] = ''

    balances = updated_balances_df # Use the LATEST balances from Cell 20

    # Get LATEST free USDT
    free_usdt = balances.loc[QUOTE_ASSET, 'Free'] if QUOTE_ASSET in balances.index else Decimal('0.0')
    print(f"Available Free {QUOTE_ASSET} (Updated): {free_usdt:.{DISPLAY_PRECISION_QTY}f}")

    cumulative_buy_cost = Decimal('0.0') # Track cost for updated check

    # Iterate through the IDEAL trades (sorted SELLs then BUYs from Cell 16)
    for index, trade in filtered_df.iterrows():
        asset = trade['Asset']; action = trade['Action']
        # Ensure data types are Decimal for calculation
        try:
            ideal_qty = Decimal(str(trade['IdealQuantity']))
            ideal_value = Decimal(str(trade['IdealValueUSDT']))
            if ideal_qty <= 0 or ideal_value <=0:
                 raise ValueError("Ideal quantity or value is zero/negative")
        except (ValueError, TypeError, InvalidOperation) as dec_err:
             print(f"  Error converting trade data for {asset} to Decimal: {dec_err}. Marking infeasible.")
             filtered_df.loc[index, 'Feasible_Updated'] = False
             filtered_df.loc[index, 'Feasibility_Reason'] = 'Invalid trade data'
             continue # Skip to next trade

        # Check SELL Feasibility vs UPDATED Free balance
        if action == 'SELL':
            free_balance = balances.loc[asset,'Free'] if asset in balances.index else Decimal('0.0')
            if ideal_qty > free_balance:
                filtered_df.loc[index, 'Feasible_Updated'] = False
                reason = f'Insufficient Free {asset} ({free_balance:.{DISPLAY_PRECISION_QTY}f} < {ideal_qty:.{DISPLAY_PRECISION_QTY}f})'
                filtered_df.loc[index, 'Feasibility_Reason'] = reason
                print(f"  Marking SELL {asset} infeasible: {reason}")


        # Check BUY Feasibility vs UPDATED Free USDT (Cumulative)
        elif action == 'BUY':
            required_usdt = ideal_value
            if cumulative_buy_cost + required_usdt > free_usdt:
                filtered_df.loc[index, 'Feasible_Updated'] = False
                reason = f'Insufficient Free {QUOTE_ASSET} (Need {required_usdt:.{DISPLAY_PRECISION_VAL}f}, Available {(free_usdt - cumulative_buy_cost):.{DISPLAY_PRECISION_VAL}f})'
                filtered_df.loc[index, 'Feasibility_Reason'] = reason
                print(f"  Marking BUY {asset} infeasible: {reason}")

            else:
                # Only increment cumulative cost if this BUY trade passes the check
                cumulative_buy_cost += required_usdt

    # Filter DataFrame based on the new Feasible_Updated column
    final_trades_to_format_df = filtered_df[filtered_df['Feasible_Updated']].copy()

    # --- Report Summary ---\
    proceed_count = len(final_trades_to_format_df)
    original_count = len(trades_df) # Count from Cell 16's output DF
    skipped_count = original_count - proceed_count

    print(f"\nFinal Feasibility Check Summary (using updated balances):")
    print(f"  {proceed_count} / {original_count} ideal trades are feasible.")
    if skipped_count > 0:
        print(f"  {skipped_count} ideal trades skipped due to updated balance constraints.")
        # Optional: Display skipped trades and reasons if needed for debugging
        # skipped_trades = filtered_df[~filtered_df['Feasible_Updated']]
        # print("--- Skipped Trades ---")
        # print(skipped_trades[['Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT', 'Feasibility_Reason']].to_string(index=False))

    if not final_trades_to_format_df.empty:
        print("\n--- Trades Passing Final Feasibility (Ready for Formatting) ---")
        display_cols = ['Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT', 'CalcPrice']
        # Formatters for display
        def fmt_qty(x): return f"{x:.{DISPLAY_PRECISION_QTY}f}" if isinstance(x, (Decimal, float)) else str(x)
        def fmt_val(x): return f"{x:.{DISPLAY_PRECISION_VAL}f}" if isinstance(x, (Decimal, float)) else str(x)
        def fmt_price(x): return f"{x:.4f}" if isinstance(x, (Decimal, float)) else str(x)
        formatters = {'IdealQuantity':fmt_qty, 'IdealValueUSDT':fmt_val, 'CalcPrice':fmt_price}
        print(final_trades_to_format_df[display_cols].to_string(formatters=formatters, index=False))
        print("\n✅ Final filtering complete. Feasible trades stored in 'final_trades_to_format_df'.")
    else:
        print("\n✅ Final filtering complete. No ideal trades were feasible with updated balances.")


# Ensure the output DataFrame exists, even if empty
if 'final_trades_to_format_df' not in locals():
     final_trades_to_format_df = pd.DataFrame()

ℹ️ Ideal trades list ('trades_df') is empty. No trades to filter.


In [26]:
# Cell 22: Format Orders for Execution

import pandas as pd
import os
import numpy as np
# Need Client for exchange info and ticker/depth, client should be initialized
from binance.client import Client
from decimal import Decimal, ROUND_DOWN, InvalidOperation
import time # For potential small delays

# --- Configuration ---\
QUOTE_ASSET = 'USDT'
# Strategy: Aim for MAKER fees by placing slightly inside the spread or at the best bid/ask
# 'BID' for BUYS, 'ASK' for SELLS. Adjust price if needed just before placement.
ORDER_PLACEMENT_STRATEGY = {'BUY': 'BID', 'SELL': 'ASK'}
# Price adjustment tolerance: If our calculated price is within this % of the
# target bid/ask, we'll use our calculated price. Otherwise, adjust to current bid/ask.
# Set to 0 to always adjust to the latest bid/ask. Set higher (e.g., 0.1) to allow slight deviation.
PRICE_ADJUSTMENT_TOLERANCE_PCT = Decimal('0.05') # e.g., 0.05%

# --- Check Prerequisites ---\
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client not initialized.")
if 'format_price_correctly' not in locals() or 'adjust_quantity_to_step' not in locals():
    raise RuntimeError("Formatting helper functions (format_price_correctly, adjust_quantity_to_step) not defined in Cell 1.")
# Use the trades that passed the FINAL feasibility check in Cell 21
if 'final_trades_to_format_df' not in locals():
    print("⚠️ Filtered trades ('final_trades_to_format_df') not found. Cannot format.")
    final_executable_orders = [] # Output list
    formatting_errors = 0 # <<< INITIALIZE HERE
elif final_trades_to_format_df.empty:
    print("ℹ️ Filtered trades list ('final_trades_to_format_df') is empty. No orders to format.")
    final_executable_orders = [] # Output list
    formatting_errors = 0 # <<< INITIALIZE HERE
else:
    print("\n--- Formatting Final Executable Orders ---")
    print(f"Strategy: Place BUY at BID, SELL at ASK (with final price check/adjustment).")

    # --- Get Exchange Info (Once per run) ---
    print("Fetching exchange information for formatting rules...")
    symbols_info = None
    try:
        # Consider caching this if running in a loop later
        exchange_info = client.get_exchange_info()
        symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
        print("✅ Exchange information received.")
    except Exception as e:
        print(f"❌ Error fetching exchange info: {e}. Formatting may be inaccurate or fail.")
        # Decide if we should stop or try to continue without rules
        # For safety, let's stop if we can't get rules.
        raise RuntimeError(f"Failed to fetch exchange info: {e}")

    # Initialize list and error counter INSIDE the else block where processing happens
    final_executable_orders = [] # List for final API order parameters
    formatting_errors = 0

    # Iterate through the FEASIBLE trades from Cell 21
    for index, trade in final_trades_to_format_df.iterrows():
        asset = trade['Asset']
        action = trade['Action'] # BUY or SELL
        symbol = f"{asset}{QUOTE_ASSET}" if asset != QUOTE_ASSET else None # Determine symbol

        # Skip if USDT (no pair to trade) or if somehow action is invalid
        if asset == QUOTE_ASSET or action not in ORDER_PLACEMENT_STRATEGY:
             print(f"  Skipping trade for {asset} (Quote asset or invalid action '{action}').")
             continue

        print(f"\nProcessing {action} {asset} (Symbol: {symbol})")

        # --- Get Formatting Rules ---
        if symbols_info is None or symbol not in symbols_info:
            print(f"  ⚠️ Rules not found for {symbol}. Skipping formatting for this trade.")
            formatting_errors += 1
            continue
        symbol_info = symbols_info[symbol]
        filters = {f['filterType']: f for f in symbol_info['filters']}
        price_filter = filters.get('PRICE_FILTER', {})
        lot_size_filter = filters.get('LOT_SIZE', {})
        # Handle both NOTIONAL and MIN_NOTIONAL filter types
        notional_filter = filters.get('NOTIONAL', filters.get('MIN_NOTIONAL', {}))
        tick_size = price_filter.get('tickSize')
        step_size = lot_size_filter.get('stepSize')
        min_notional_str = notional_filter.get('minNotional', '0.0') # Default to 0 if missing

        if not tick_size or not step_size:
            print(f"  ⚠️ Missing PRICE_FILTER(tickSize) or LOT_SIZE(stepSize) for {symbol}. Skipping.")
            formatting_errors += 1
            continue
        try:
            min_notional = Decimal(min_notional_str)
        except InvalidOperation:
             print(f"  ⚠️ Invalid minNotional value '{min_notional_str}' for {symbol}. Skipping.")
             formatting_errors += 1
             continue

        # --- Get Data from DataFrame ---
        try:
            # Use the quantity calculated in Cell 16, stored in final_trades_to_format_df
            ideal_quantity_dec = Decimal(str(trade['IdealQuantity']))
            # Use the price from Cell 16 as an initial reference if needed, but we'll fetch live price
            # reference_price_dec = Decimal(str(trade['CalcPrice'])) # Optional: use for comparison
        except (ValueError, TypeError, InvalidOperation) as dec_err:
             print(f"  Error converting trade data for {symbol} to Decimal: {dec_err}. Skipping.")
             formatting_errors += 1
             continue

        if ideal_quantity_dec <= 0:
             print(f"  Skipping {action} {asset}: Ideal quantity is zero or negative.")
             continue

        # --- Fetch Live Order Book Price (Best Bid/Ask) ---
        live_price_to_use = None
        try:
            depth = client.get_order_book(symbol=symbol, limit=5) # Get top 5 levels
            if action == 'BUY' and depth['bids']:
                live_price_to_use = Decimal(depth['bids'][0][0]) # Best bid
                print(f"  Live Best Bid: {live_price_to_use}")
            elif action == 'SELL' and depth['asks']:
                live_price_to_use = Decimal(depth['asks'][0][0]) # Best ask
                print(f"  Live Best Ask: {live_price_to_use}")
            else:
                print(f"  ⚠️ Could not get live {'bid' if action == 'BUY' else 'ask'} price for {symbol}. Trying ticker...")
                # Fallback to ticker if depth fails
                ticker = client.get_symbol_ticker(symbol=symbol)
                live_price_to_use = Decimal(ticker['price'])
                print(f"  Using Ticker Price as fallback: {live_price_to_use}")

            if live_price_to_use is None or live_price_to_use <= 0:
                 raise ValueError("Live price is invalid")

        except Exception as e:
            print(f"  ❌ Error fetching live price for {symbol}: {e}. Skipping.")
            formatting_errors += 1
            continue

        # --- Format Price and Quantity ---
        try:
            # Format the LIVE price according to tickSize
            adjusted_price_str = format_price_correctly(live_price_to_use, tick_size)
            adjusted_price_dec = Decimal(adjusted_price_str) # Decimal version for calculations

            # Adjust the IDEAL quantity based on step_size
            adjusted_quantity_str = adjust_quantity_to_step(ideal_quantity_dec, step_size)
            adjusted_quantity_dec = Decimal(adjusted_quantity_str) # Decimal version

            print(f"  Formatted Price Str: {adjusted_price_str} (Based on live {ORDER_PLACEMENT_STRATEGY[action]})")
            print(f"  Adjusted Quantity Str: {adjusted_quantity_str}")

        except Exception as fmt_err:
            print(f"  ❌ Formatting error (Price/Qty): {fmt_err}. Skipping.")
            formatting_errors += 1
            continue

        # --- Final Checks: Quantity > 0 and Min Notional ---
        if adjusted_quantity_dec <= 0:
            print(f"  ⚠️ Adjusted quantity is zero or less ({adjusted_quantity_str}). Skipping.")
            continue # Skip if quantity becomes zero after adjustment

        order_value_final = adjusted_quantity_dec * adjusted_price_dec
        if min_notional > 0 and order_value_final < min_notional:
             print(f"  ⚠️ Final order value ({order_value_final:.8f}) is less than min notional ({min_notional:.8f}). Skipping.")
             # Optional: Could try adjusting quantity upwards slightly to meet min notional,
             # but this adds complexity. Skipping is safer for now.
             continue

        print(f"  ✅ Final order meets min notional ({order_value_final:.8f} >= {min_notional:.8f}).")

        # --- Add Validated Order Parameters to List ---
        final_executable_orders.append({
            'symbol': symbol,
            'side': action.upper(), # Ensure uppercase ('BUY' or 'SELL')
            'type': 'LIMIT',
            'timeInForce': 'GTC', # Good Till Cancelled
            'quantity': adjusted_quantity_str, # Use FORMATTED STRING
            'price': adjusted_price_str       # Use FORMATTED STRING
        })
        print(f"  ✅ Added valid LIMIT order parameters for {symbol} to final execution list.")

        # Optional small delay between processing symbols if hitting rate limits during formatting
        # time.sleep(0.1)


# --- Final Display ---
print("\n--- Final Orders Ready for Confirmation ---")
# formatting_errors is guaranteed to exist here because it's initialized in both branches of the initial if/else
if not final_executable_orders:
    if formatting_errors > 0:
         print(f"No orders ready due to {formatting_errors} formatting error(s).")
    else:
         print("No feasible trades required formatting.") # Message when final_trades_to_format_df was empty
else:
    executable_orders_df = pd.DataFrame(final_executable_orders)
    print(executable_orders_df[['symbol', 'side', 'type', 'quantity', 'price']])
    print(f"\nStored {len(final_executable_orders)} order parameter set(s) in 'final_executable_orders'.")

# Always print formatting error summary if any occurred
if formatting_errors > 0:
     print(f"⚠️ Encountered {formatting_errors} error(s) during formatting.")


# Ensure the list exists even if empty
if 'final_executable_orders' not in locals():
     final_executable_orders = []

ℹ️ Filtered trades list ('final_trades_to_format_df') is empty. No orders to format.

--- Final Orders Ready for Confirmation ---
No feasible trades required formatting.


In [27]:
# Cell 23: Final LLM Check & User Confirmation

import pandas as pd
import json
import time
from decimal import Decimal
try:
    import google.generativeai as genai
    # Check if it was configured earlier, assume False if variable doesn't exist
    if 'genai_configured' not in locals(): genai_configured = False
except ImportError:
    print("Warning: google.generativeai not found.")
    genai = None
    genai_configured = False # Ensure flag is False if import fails
from dotenv import load_dotenv # Usually good practice to load_dotenv again just in case

# --- Configuration ---\
# EXECUTE_REAL_ORDERS_FLAG is determined by user input later if REQUIRE_CONFIRMATION is True
REQUIRE_CONFIRMATION = True # Set to False only for fully automated testing (Use Caution!)
PERFORM_LLM_FINAL_CHECK = False # Keep OFF for now - reliability issues previously noted
LLM_MODEL_NAME_FINAL_CHECK = "gemini-1.5-flash-latest"
QUOTE_ASSET = 'USDT'
RSI_PERIOD = 14 # Used if pulling TA data for LLM prompt
DEFAULT_LLM_REC = "APPROVE" # Default if check skipped/fails/disabled

# --- Initialize flags ---\
proceed_user_confirmation = False # Final user decision flag
llm_final_recommendation = DEFAULT_LLM_REC # Default recommendation
llm_final_rationale = "LLM check skipped or default." # Default rationale

# --- Check Prerequisites --- #
execution_possible = False # Can we even execute anything?
llm_check_possible = PERFORM_LLM_FINAL_CHECK # Should we try the LLM check?

# Basic check for executable orders list
if 'final_executable_orders' not in locals():
    print("⚠️ 'final_executable_orders' list not found (Cell 22 likely failed or skipped).")
elif not isinstance(final_executable_orders, list):
     print("⚠️ 'final_executable_orders' is not a list.")
elif not final_executable_orders:
    print("ℹ️ 'final_executable_orders' is empty. Nothing to execute.")
else:
    # Only possible to execute if the list exists, is a list, and is not empty
    execution_possible = True

# Check LLM prerequisites only if enabled AND execution is possible
if not execution_possible:
    llm_check_possible = False # No point checking if we can't execute
elif PERFORM_LLM_FINAL_CHECK and not genai_configured:
     print("⚠️ LLM not configured. Disabling final LLM check.")
     llm_check_possible = False
elif PERFORM_LLM_FINAL_CHECK and 'full_market_state' not in locals():
     print("⚠️ Market state ('full_market_state') not found. Disabling final LLM check.")
     llm_check_possible = False
elif PERFORM_LLM_FINAL_CHECK and 'news_summary' not in locals():
     print("⚠️ News summary ('news_summary') not found. Disabling final LLM check.")
     llm_check_possible = False
# --- END Check Prerequisites --- #

# --- Main Logic ---
if not execution_possible:
     print("\nSkipping confirmation and execution steps as no orders are ready or prerequisites missing.")
else:
    # --- Display Orders First ---
    print("\n" + "="*60)
    print(" !!! FINAL CONFIRMATION BEFORE LIVE EXECUTION !!!")
    print("="*60)
    print("The following order(s) will be placed if confirmed:")
    try:
        orders_to_exec_df = pd.DataFrame(final_executable_orders)
        # Select columns that definitely exist based on Cell 22's output format
        display_cols = ['symbol', 'side', 'type', 'quantity', 'price']
        print(orders_to_exec_df[display_cols])
        print(f"\nTotal order(s) to execute: {len(orders_to_exec_df)}")
    except Exception as e:
        print(f"Error previewing orders: {e}\nRaw list:\n{final_executable_orders}")
    print("="*60)

    # --- Perform Final LLM Sanity Check (if possible) ---
    if llm_check_possible:
        print("\n--- Performing Final LLM Sanity Check ---")
        try:
            # Prepare Context (Simplified)
            orders_str_list = []
            symbols_involved = set()
            for order in final_executable_orders:
                orders_str_list.append(f"- {order['side']} {order['quantity']} {order['symbol']} @ {order['price']}")
                symbols_involved.add(order['symbol'])
            orders_for_prompt = "\n".join(orders_str_list)

            # Simplified TA Summary for prompt
            ta_summary_parts = []
            for sym in symbols_involved:
                 state = full_market_state.get(sym)
                 if state and not state.get('error'):
                     # Example: Just include RSI if available
                     rsi_val = state.get(f'last_{PRICE_SOURCE_INTERVAL}_rsi') # Assuming PRICE_SOURCE_INTERVAL matches TA interval needed
                     rsi_str = f"RSI {rsi_val:.1f}" if rsi_val is not None else "RSI N/A"
                     ta_summary_parts.append(f"{sym}: {rsi_str}")
                 else: ta_summary_parts.append(f"{sym}: TA N/A")
            ta_summary_for_prompt = "; ".join(ta_summary_parts) if ta_summary_parts else "N/A"

            news_context_summary = news_summary.get('summary', 'N/A')

            # Construct Simplified Final Check Prompt ("Approve/Reject Batch?")
            final_check_prompt = f"""
            Reviewing a batch of automated cryptocurrency trades before execution.

            Strategy: Long-term rebalancing towards target allocations, aiming for maker fees.
            News Context: {news_context_summary}
            TA Snippet: {ta_summary_for_prompt}

            Proposed Orders:
            {orders_for_prompt}

            Task: Based ONLY on the information provided, do you see any glaring contradictions or red flags that warrant halting this specific batch? Respond ONLY with:
            **APPROVE EXECUTION**
            OR
            **REJECT EXECUTION**
            Optionally add a brief comment AFTER the keyword line.
            """

            # Call LLM
            load_dotenv() # Ensure API key is loaded
            model = genai.GenerativeModel(LLM_MODEL_NAME_FINAL_CHECK)
            safety_settings=[{"category": c, "threshold": "BLOCK_NONE"} for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]]
            print("Sending simplified final check request to LLM...")
            response = model.generate_content(final_check_prompt, safety_settings=safety_settings)

            if not response.candidates or (response.prompt_feedback and response.prompt_feedback.block_reason):
                 reason = response.prompt_feedback.block_reason if response.prompt_feedback else "Unknown"
                 print(f"LLM Final Check BLOCKED: {reason}")
                 llm_final_recommendation = "REJECT"
                 llm_final_rationale = f"LLM Response Blocked: {reason}"
            else:
                 raw_text = response.text.strip()
                 print(f"LLM Final Check Response:\n---\n{raw_text}\n---")
                 llm_final_rationale = raw_text
                 # Check first line primarily for the keyword
                 first_line_upper = raw_text.split('\n')[0].upper().strip()
                 if "**REJECT EXECUTION**" in first_line_upper:
                     llm_final_recommendation = "REJECT"
                     print("LLM signaled REJECT.")
                 elif "**APPROVE EXECUTION**" in first_line_upper:
                     llm_final_recommendation = "APPROVE"
                     print("LLM signaled APPROVE.")
                 else:
                      # If keyword not clear in first line, default to REJECT for safety
                      llm_final_recommendation = "REJECT"
                      llm_final_rationale += " (Keyword unclear)"
                      print("⚠️ LLM recommendation keyword unclear, defaulting to REJECT.")

        except Exception as llm_err:
            print(f"❌ Error during LLM final check: {llm_err}")
            llm_final_recommendation = "REJECT" # Default to REJECT on error
            llm_final_rationale = f"LLM API Error: {llm_err}"

        print("-" * 30); print(f"LLM FINAL RECOMMENDATION: {llm_final_recommendation}"); print("-" * 30)
    # Handle cases where check was disabled or skipped
    elif PERFORM_LLM_FINAL_CHECK: # Check was enabled but skipped due to other prereqs
        print("\n--- Skipping Final LLM Sanity Check (Prerequisites Missing) ---")
        llm_final_recommendation = DEFAULT_LLM_REC # Use default
        llm_final_rationale = "LLM check skipped due to missing prerequisites."
    else: # Check was disabled via config
        print("\n--- Skipping Final LLM Sanity Check (Setting: OFF) ---")
        llm_final_recommendation = DEFAULT_LLM_REC # Use default
        llm_final_rationale = "LLM check disabled by configuration."


    # --- Get User Confirmation ---
    if REQUIRE_CONFIRMATION:
        print("\nDecision Point: Review the prepared order(s) and LLM recommendation above.")
        try:
            # Loop until valid input or EOF
            while True:
                prompt_msg = f">>> Type 'yes' to EXECUTE, 'no' to CANCEL (LLM: {llm_final_recommendation}): "
                confirm = input(prompt_msg).lower().strip()
                if confirm == 'yes':
                    proceed_user_confirmation = True
                    print("\nUser confirmed execution...")
                    break
                elif confirm == 'no':
                    proceed_user_confirmation = False
                    print("\nUser cancelled execution.")
                    break
                else:
                    print("  Invalid input. Please type 'yes' or 'no'.")
        except EOFError:
            # If input stream is closed (e.g., running non-interactively)
            print("\nCould not get user confirmation (EOFError). EXECUTION CANCELLED.")
            proceed_user_confirmation = False
    else: # Confirmation not required
        print("\nSkipping user confirmation step (REQUIRE_CONFIRMATION = False).")
        # Automatically proceed unless LLM rejected (and LLM check was performed)
        if llm_check_possible and llm_final_recommendation == "REJECT":
             print("EXECUTION HALTED: LLM recommended REJECT (and user confirmation was skipped).")
             proceed_user_confirmation = False
        else:
             print("Auto-proceeding with execution (or lack thereof if no orders).")
             proceed_user_confirmation = True


# --- Final Status ---
# This flag now reflects the final decision based on order existence, LLM check (if applicable), and user input (if applicable).
print(f"\n✅ Confirmation step complete. Ready to proceed to execution module? {proceed_user_confirmation}")

# Ensure flags exist globally
if 'proceed_user_confirmation' not in locals(): proceed_user_confirmation = False
if 'llm_final_recommendation' not in locals(): llm_final_recommendation = "SKIP"
if 'llm_final_rationale' not in locals(): llm_final_rationale = "Not generated."

ℹ️ 'final_executable_orders' is empty. Nothing to execute.

Skipping confirmation and execution steps as no orders are ready or prerequisites missing.

✅ Confirmation step complete. Ready to proceed to execution module? False
